In [18]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import warnings
import tensorflow as tf
from time import time
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from tffm import TFFMClassifier
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from sklearn import preprocessing
from random import random
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
%matplotlib inline

In [9]:
# preprocessing
data = pd.read_csv(r'...\data_net.csv', sep = ';')
data.drop('Unnamed: 0', axis = 1,inplace = True)

In [10]:
# features & labels
labels = data['Click']
features = data.drop('Click', axis = 1)
features

,Browsname,Searcher,TypeCon,Country,deviceType,Model,ModelCompany,NewID,Reversed,Type,System,FirstSearcher,Version,Sex,Age,TimeSpent,InternalCode,id_transf
0,Chrome,Google,wi-fi,ita,SmartPhone,Nokia 2240,Nokia,yes,no,mobile,Android,Yahoo,5.0,male,21,1.020,fergie,25465885
1,Firefox,Yahoo,cable,fra,SmartPhone,Meizu 4 pro,Meizu,yes,no,TV,Android,Yahoo,4.2,female,34,0.123,krag,65458971
2,Chrome,Yandex,wi-fi,rus,SmartPhone,Iphone 6,Iphone,yes,yes,mobile,Ios,Yahoo,9.0,male,53,0.140,leslie,21547895
3,IE,Google,cable,usa,SmartPhone,Iphone X,Iphone,no,no,web,Ios,Yahoo,9.0,male,30,0.010,fergie,32541568
4,IE,Google,wi-fi,usa,SmartPhone,Galaxy J4,SAMSUNG,no,no,TV,Android,Yahoo,5.5.1,female,15,1.100,frent,12023515


In [12]:
# create the dictionary of unique attributes' lists
unique_attr = {}
for attr in features.columns:
    unique_attr[attr] = features[attr].unique().tolist()

In [ ]:
# OneHot encoder
encoder = preprocessing.OneHotEncoder(categories=[unique_attr[i] for i in unique_attr], sparse = False, handle_unknown='ignore')
features['CONCAT'] = features.values.tolist()
features['CONCAT'].head()

In [4]:
# encoding into the list
t0 = time()
features['ENCODED'] = [encoder.fit_transform([i]).flatten() for i in features['CONCAT']]
print ("time on encoding:", round(time()-t0, 3), "s")

time on encoding: 50.699 s


In [5]:
# create the array with feature vectors
features_list = [list(i) for i in features['ENCODED']]
features_list_array = np.array(features_list)
# create the array with label vector 
labels_list_array = np.array(labels.tolist())

In [7]:
# cross-validation
X_train, X_test, y_train, y_test = train_test_split(features_list_array, labels_list_array, random_state=35, test_size=0.2)

In [8]:
# Testing different classifiers
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier(silent=False)],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.93      0.98      0.96      9255
         1.0       0.31      0.11      0.16       745

   micro avg       0.92      0.92      0.92     10000
   macro avg       0.62      0.54      0.56     10000
weighted avg       0.89      0.92      0.90     10000

[[9075  180]
 [ 665   80]]
Accuracy is  91.55
Time on model's work: 7.456 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      9255
         1.0       0.98      0.06      0.12       745

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.95      0.53      0.54     10000
weighted avg       0.93      0.93      0.90     10000

[[9254    1]
 [ 699   46]]
Accuracy is  93.0
Time on model's work: 476.008 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.93      0.98      0.96      9255
         1.0       0.30      0.11      0.16       745

   micro avg       0.91      0.91      0.91     10000
   macro avg       0.62      0.55      0.56     10000
weighted avg       0.89      0.91      0.90     10000

[[9065  190]
 [ 662   83]]
Accuracy is  91.47999999999999
Time on model's work: 14.262 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      9255
         1.0       0.98      0.06      0.11       745

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.95      0.53      0.54     10000
weighted avg       0.93      0.93      0.90     10000

[[9254    1]
 [ 700   45]]
Accuracy is  92.99
Time on model's work: 105.722 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95      9255
         1.0

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

933:	learn: 0.1963884	total: 4m 52s	remaining: 20.7s
934:	learn: 0.1963722	total: 4m 52s	remaining: 20.4s
935:	learn: 0.1963565	total: 4m 53s	remaining: 20s
936:	learn: 0.1963475	total: 4m 53s	remaining: 19.7s
937:	learn: 0.1963354	total: 4m 53s	remaining: 19.4s
938:	learn: 0.1963329	total: 4m 54s	remaining: 19.1s
939:	learn: 0.1963304	total: 4m 54s	remaining: 18.8s
940:	learn: 0.1963191	total: 4m 54s	remaining: 18.5s
941:	learn: 0.1963139	total: 4m 55s	remaining: 18.2s
942:	learn: 0.1963082	total: 4m 55s	remaining: 17.9s
943:	learn: 0.1962885	total: 4m 55s	remaining: 17.5s
944:	learn: 0.1962817	total: 4m 56s	remaining: 17.2s
945:	learn: 0.1962779	total: 4m 56s	remaining: 16.9s
946:	learn: 0.1962754	total: 4m 56s	remaining: 16.6s
947:	learn: 0.1962630	total: 4m 57s	remaining: 16.3s
948:	learn: 0.1962613	total: 4m 57s	remaining: 16s
949:	learn: 0.1962502	total: 4m 57s	remaining: 15.7s
950:	learn: 0.1962395	total: 4m 58s	remaining: 15.4s
951:	learn: 0.1962278	total: 4m 58s	remaining: 15s

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      9255
         1.0       0.98      0.06      0.12       745

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.95      0.53      0.54     10000
weighted avg       0.93      0.93      0.90     10000

[[9254    1]
 [ 699   46]]
Accuracy is  93.0
Time on model's work: 1.156 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      9255
         1.0       1.00      0.06      0.11       745

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.96      0.53      0.54     10000
weighted avg       0.93      0.93      0.90     10000

[[9255    0]
 [ 700   45]]
Accuracy is  93.0
Time on model's work: 0.846 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      9255
         1.0       0.65      0.05      0.09       745

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.79      0.52      0.53     10000
weighted avg       0.91      0.93      0.90     10000

[[9235   20]
 [ 708   37]]
Accuracy is  92.72
Time on model's work: 137.038 s
TOTAL TIME SPENT:  2634.366 s


All models were not able to predict imbalanced clas '1' correctly. Let's use some methods to deal with imbalanced classes.

### 1. Resampling variant
a) delete instances from the over-represented class - under-sampling

In [3]:
# create 2 dataframe with all clicked rows and select the same number of rows as in '1' from '0' data 
data_1 = data[data['Is_lp_click'] == 1]
data_0 = data[data['Is_lp_click'] == 0].sample(35059, random_state = 3)
# concat 2 dataframes and shuffle it
data_concat = pd.concat([data_1, data_0])
data_under = data_concat.sample(frac=1)

In [5]:
# features & labels
labels = data_under['Is_lp_click']
features = data_under.drop('Is_lp_click', axis = 1)
# create the dictionary of unique attributes' lists
unique_attr = {}
for attr in features.columns:
    unique_attr[attr] = features[attr].unique().tolist()
unique_attr
# features encoding
encoder = preprocessing.OneHotEncoder(categories=[unique_attr[i] for i in unique_attr], sparse = False, handle_unknown='ignore')
encoder
features['CONCAT'] = features.values.tolist()
features['CONCAT'].head()
t0 = time()
features['ENCODED'] = [encoder.fit_transform([i]).flatten() for i in features['CONCAT']]
print ("time on encoding:", round(time()-t0, 3), "s")
# create the array with feature vectors
features_list = [list(i) for i in features['ENCODED']]
features_list_array = np.array(features_list)
# create the array with label vector 
labels_list_array = np.array(labels.tolist())

time on encoding: 69.695 s


In [6]:
X_train, X_test, y_train, y_test = train_test_split(features_list_array, labels_list_array, random_state=35, test_size=0.2)

In [7]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.75      0.69      0.72      6986
         1.0       0.71      0.77      0.74      7038

   micro avg       0.73      0.73      0.73     14024
   macro avg       0.73      0.73      0.73     14024
weighted avg       0.73      0.73      0.73     14024

[[4825 2161]
 [1624 5414]]
Accuracy is  73.01055333713634
Time on model's work: 20.163 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.89      0.62      0.73      6986
         1.0       0.71      0.92      0.80      7038

   micro avg       0.77      0.77      0.77     14024
   macro avg       0.80      0.77      0.76     14024
weighted avg       0.80      0.77      0.76     14024

[[4304 2682]
 [ 537 6501]]
Accuracy is  77.04649172846548
Time on model's work: 736.97 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.72      0.71      0.72      6986
         1.0       0.72      0.73      0.72      7038

   micro avg       0.72      0.72      0.72     14024
   macro avg       0.72      0.72      0.72     14024
weighted avg       0.72      0.72      0.72     14024

[[4948 2038]
 [1895 5143]]
Accuracy is  71.95521962350256
Time on model's work: 23.708 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.87      0.62      0.73      6986
         1.0       0.71      0.91      0.80      7038

   micro avg       0.77      0.77      0.77     14024
   macro avg       0.79      0.77      0.76     14024
weighted avg       0.79      0.77      0.76     14024

[[4354 2632]
 [ 633 6405]]
Accuracy is  76.71848260125499
Time on model's work: 139.741 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.75      0.68      0.72      6986


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

932:	learn: 0.4734175	total: 4m 26s	remaining: 19.2s
933:	learn: 0.4733987	total: 4m 27s	remaining: 18.9s
934:	learn: 0.4733912	total: 4m 27s	remaining: 18.6s
935:	learn: 0.4733775	total: 4m 27s	remaining: 18.3s
936:	learn: 0.4733671	total: 4m 28s	remaining: 18s
937:	learn: 0.4733570	total: 4m 28s	remaining: 17.7s
938:	learn: 0.4733511	total: 4m 28s	remaining: 17.5s
939:	learn: 0.4733459	total: 4m 29s	remaining: 17.2s
940:	learn: 0.4733149	total: 4m 29s	remaining: 16.9s
941:	learn: 0.4733088	total: 4m 29s	remaining: 16.6s
942:	learn: 0.4732921	total: 4m 30s	remaining: 16.3s
943:	learn: 0.4732793	total: 4m 30s	remaining: 16s
944:	learn: 0.4732761	total: 4m 30s	remaining: 15.8s
945:	learn: 0.4732722	total: 4m 30s	remaining: 15.5s
946:	learn: 0.4732610	total: 4m 31s	remaining: 15.2s
947:	learn: 0.4732488	total: 4m 31s	remaining: 14.9s
948:	learn: 0.4732428	total: 4m 31s	remaining: 14.6s
949:	learn: 0.4732351	total: 4m 32s	remaining: 14.3s
950:	learn: 0.4732288	total: 4m 32s	remaining: 14s

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.86      0.63      0.73      6986
         1.0       0.71      0.90      0.80      7038

   micro avg       0.77      0.77      0.77     14024
   macro avg       0.79      0.77      0.76     14024
weighted avg       0.79      0.77      0.76     14024

[[4427 2559]
 [ 693 6345]]
Accuracy is  76.81118083285796
Time on model's work: 3.187 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.56      0.92      0.70      6986
         1.0       0.78      0.28      0.41      7038

   micro avg       0.60      0.60      0.60     14024
   macro avg       0.67      0.60      0.55     14024
weighted avg       0.67      0.60      0.55     14024

[[6432  554]
 [5066 1972]]
Accuracy is  59.92584141471763
Time on model's work: 1.925 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.86      0.63      0.73      6986
         1.0       0.71      0.89      0.79      7038

   micro avg       0.76      0.76      0.76     14024
   macro avg       0.78      0.76      0.76     14024
weighted avg       0.78      0.76      0.76     14024

[[4434 2552]
 [ 749 6289]]
Accuracy is  76.46177980604678
Time on model's work: 229.59 s
TOTAL TIME SPENT:  3863.884 s


In [7]:
# TFFM sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)

In [12]:
# input_type='sparse' /// rank == 10
for order in [2, 3]:
    model = TFFMClassifier(
        order=order, 
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=512,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:26<00:00,  1.93epoch/s]


[order=2] accuracy: 0.7620507701083856
[[4428 2613]
 [ 724 6259]]
              precision    recall  f1-score   support

         0.0       0.86      0.63      0.73      7041
         1.0       0.71      0.90      0.79      6983

   micro avg       0.76      0.76      0.76     14024
   macro avg       0.78      0.76      0.76     14024
weighted avg       0.78      0.76      0.76     14024



100%|██████████| 50/50 [00:49<00:00,  1.02epoch/s]


[order=3] accuracy: 0.761908157444381
[[4448 2593]
 [ 746 6237]]
              precision    recall  f1-score   support

         0.0       0.86      0.63      0.73      7041
         1.0       0.71      0.89      0.79      6983

   micro avg       0.76      0.76      0.76     14024
   macro avg       0.78      0.76      0.76     14024
weighted avg       0.78      0.76      0.76     14024



In [22]:
# FtrlOptimizer
# input_type='sparse' /// rank == 10
model = TFFMClassifier(
    order=2, 
    rank=10, 
    optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
    n_epochs=50, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()


100%|██████████| 50/50 [00:24<00:00,  2.08epoch/s]


[order=3] accuracy: 0.7507130633200229
[[4336 2705]
 [ 791 6192]]
              precision    recall  f1-score   support

         0.0       0.85      0.62      0.71      7041
         1.0       0.70      0.89      0.78      6983

   micro avg       0.75      0.75      0.75     14024
   macro avg       0.77      0.75      0.75     14024
weighted avg       0.77      0.75      0.75     14024



In [26]:
# sample_weight='balanced'
model = TFFMClassifier(
    order=2,
    sample_weight='balanced',
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=50, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()


100%|██████████| 50/50 [00:24<00:00,  2.07epoch/s]


[order=3] accuracy: 0.7623359954363947
[[4427 2614]
 [ 719 6264]]
              precision    recall  f1-score   support

         0.0       0.86      0.63      0.73      7041
         1.0       0.71      0.90      0.79      6983

   micro avg       0.76      0.76      0.76     14024
   macro avg       0.78      0.76      0.76     14024
weighted avg       0.78      0.76      0.76     14024



In [28]:
# weight - optional
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=20, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()


100%|██████████| 20/20 [00:09<00:00,  2.11epoch/s]


[order=3] accuracy: 0.7638334284084427
[[4509 2532]
 [ 780 6203]]
              precision    recall  f1-score   support

         0.0       0.85      0.64      0.73      7041
         1.0       0.71      0.89      0.79      6983

   micro avg       0.76      0.76      0.76     14024
   macro avg       0.78      0.76      0.76     14024
weighted avg       0.78      0.76      0.76     14024




100%|██████████| 20/20 [00:09<00:00,  2.02epoch/s]


[order=3] accuracy: 0.7651169423844837
[[4292 2749]
 [ 545 6438]]
              precision    recall  f1-score   support

         0.0       0.89      0.61      0.72      7041
         1.0       0.70      0.92      0.80      6983

   micro avg       0.77      0.77      0.77     14024
   macro avg       0.79      0.77      0.76     14024
weighted avg       0.79      0.77      0.76     14024




100%|██████████| 20/20 [00:09<00:00,  2.12epoch/s]


[order=3] accuracy: 0.7600541928123218
[[4155 2886]
 [ 479 6504]]
              precision    recall  f1-score   support

         0.0       0.90      0.59      0.71      7041
         1.0       0.69      0.93      0.79      6983

   micro avg       0.76      0.76      0.76     14024
   macro avg       0.79      0.76      0.75     14024
weighted avg       0.80      0.76      0.75     14024




100%|██████████| 20/20 [00:09<00:00,  2.06epoch/s]


[order=3] accuracy: 0.7501426126640046
[[3960 3081]
 [ 423 6560]]
              precision    recall  f1-score   support

         0.0       0.90      0.56      0.69      7041
         1.0       0.68      0.94      0.79      6983

   micro avg       0.75      0.75      0.75     14024
   macro avg       0.79      0.75      0.74     14024
weighted avg       0.79      0.75      0.74     14024



In [39]:
# weight - 2.0 best
model = TFFMClassifier(
    order=2,
    pos_class_weight=2.0,
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=100, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()


100%|██████████| 100/100 [00:48<00:00,  2.10epoch/s]


[order=3] accuracy: 0.7580576155162578
[[4204 2837]
 [ 556 6427]]
              precision    recall  f1-score   support

         0.0       0.88      0.60      0.71      7041
         1.0       0.69      0.92      0.79      6983

   micro avg       0.76      0.76      0.76     14024
   macro avg       0.79      0.76      0.75     14024
weighted avg       0.79      0.76      0.75     14024



In [42]:
# BEST OPTIMIZED TFFM
model = TFFMClassifier(
    order=2, 
    rank=10,
    pos_class_weight=2.0,
    optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
    n_epochs=100, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()


100%|██████████| 100/100 [00:49<00:00,  1.98epoch/s]


[order=3] accuracy: 0.7678265830005705
[[4326 2715]
 [ 541 6442]]
              precision    recall  f1-score   support

         0.0       0.89      0.61      0.73      7041
         1.0       0.70      0.92      0.80      6983

   micro avg       0.77      0.77      0.77     14024
   macro avg       0.80      0.77      0.76     14024
weighted avg       0.80      0.77      0.76     14024



In [44]:
# KERAS
model = Sequential()
model.add(Dense(128, input_dim=2454, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=256)
score = model.evaluate(X_test, y_test, batch_size=256)
print(score)

Epoch 1/20
56094/56094 [==============================] - ETA: 1:04 - loss: 0.7754 - acc: 0.511 - ETA: 19s - loss: 0.7481 - acc: 0.526 - ETA: 12s - loss: 0.7456 - acc: 0.51 - ETA: 10s - loss: 0.7361 - acc: 0.52 - ETA: 8s - loss: 0.7298 - acc: 0.5240 - ETA: 7s - loss: 0.7252 - acc: 0.532 - ETA: 6s - loss: 0.7197 - acc: 0.536 - ETA: 6s - loss: 0.7169 - acc: 0.535 - ETA: 5s - loss: 0.7160 - acc: 0.534 - ETA: 5s - loss: 0.7142 - acc: 0.536 - ETA: 5s - loss: 0.7124 - acc: 0.537 - ETA: 5s - loss: 0.7096 - acc: 0.541 - ETA: 5s - loss: 0.7055 - acc: 0.548 - ETA: 5s - loss: 0.7023 - acc: 0.551 - ETA: 4s - loss: 0.6990 - acc: 0.557 - ETA: 4s - loss: 0.6956 - acc: 0.562 - ETA: 4s - loss: 0.6932 - acc: 0.566 - ETA: 4s - loss: 0.6918 - acc: 0.568 - ETA: 4s - loss: 0.6885 - acc: 0.572 - ETA: 4s - loss: 0.6859 - acc: 0.577 - ETA: 4s - loss: 0.6835 - acc: 0.581 - ETA: 3s - loss: 0.6809 - acc: 0.585 - ETA: 3s - loss: 0.6771 - acc: 0.591 - ETA: 3s - loss: 0.6730 - acc: 0.596 - ETA: 3s - loss: 0.6710 - a

56094/56094 [==============================] - ETA: 3s - loss: 0.4765 - acc: 0.781 - ETA: 4s - loss: 0.4855 - acc: 0.785 - ETA: 3s - loss: 0.4962 - acc: 0.776 - ETA: 4s - loss: 0.4982 - acc: 0.778 - ETA: 4s - loss: 0.5023 - acc: 0.777 - ETA: 4s - loss: 0.5037 - acc: 0.776 - ETA: 3s - loss: 0.5073 - acc: 0.772 - ETA: 3s - loss: 0.5075 - acc: 0.771 - ETA: 3s - loss: 0.5098 - acc: 0.769 - ETA: 3s - loss: 0.5076 - acc: 0.771 - ETA: 3s - loss: 0.5057 - acc: 0.773 - ETA: 3s - loss: 0.5053 - acc: 0.774 - ETA: 3s - loss: 0.5064 - acc: 0.774 - ETA: 3s - loss: 0.5063 - acc: 0.773 - ETA: 3s - loss: 0.5065 - acc: 0.771 - ETA: 3s - loss: 0.5063 - acc: 0.771 - ETA: 3s - loss: 0.5055 - acc: 0.772 - ETA: 3s - loss: 0.5070 - acc: 0.771 - ETA: 3s - loss: 0.5072 - acc: 0.771 - ETA: 3s - loss: 0.5061 - acc: 0.772 - ETA: 3s - loss: 0.5055 - acc: 0.772 - ETA: 3s - loss: 0.5065 - acc: 0.772 - ETA: 3s - loss: 0.5068 - acc: 0.772 - ETA: 3s - loss: 0.5058 - acc: 0.773 - ETA: 2s - loss: 0.5064 - acc: 0.772 - ETA

56094/56094 [==============================] - ETA: 3s - loss: 0.4813 - acc: 0.769 - ETA: 3s - loss: 0.4986 - acc: 0.763 - ETA: 4s - loss: 0.4847 - acc: 0.770 - ETA: 4s - loss: 0.4909 - acc: 0.771 - ETA: 4s - loss: 0.4920 - acc: 0.770 - ETA: 4s - loss: 0.4912 - acc: 0.770 - ETA: 3s - loss: 0.4892 - acc: 0.771 - ETA: 3s - loss: 0.4903 - acc: 0.770 - ETA: 3s - loss: 0.4909 - acc: 0.771 - ETA: 3s - loss: 0.4940 - acc: 0.771 - ETA: 3s - loss: 0.4920 - acc: 0.773 - ETA: 3s - loss: 0.4921 - acc: 0.772 - ETA: 3s - loss: 0.4927 - acc: 0.771 - ETA: 3s - loss: 0.4907 - acc: 0.772 - ETA: 3s - loss: 0.4921 - acc: 0.772 - ETA: 3s - loss: 0.4935 - acc: 0.771 - ETA: 3s - loss: 0.4915 - acc: 0.772 - ETA: 3s - loss: 0.4927 - acc: 0.771 - ETA: 3s - loss: 0.4936 - acc: 0.771 - ETA: 2s - loss: 0.4935 - acc: 0.771 - ETA: 2s - loss: 0.4939 - acc: 0.770 - ETA: 2s - loss: 0.4940 - acc: 0.770 - ETA: 2s - loss: 0.4935 - acc: 0.771 - ETA: 2s - loss: 0.4929 - acc: 0.771 - ETA: 2s - loss: 0.4941 - acc: 0.769 - ETA

56094/56094 [==============================] - ETA: 3s - loss: 0.5410 - acc: 0.738 - ETA: 3s - loss: 0.5108 - acc: 0.764 - ETA: 3s - loss: 0.4974 - acc: 0.770 - ETA: 2s - loss: 0.5044 - acc: 0.765 - ETA: 2s - loss: 0.5053 - acc: 0.762 - ETA: 2s - loss: 0.4976 - acc: 0.766 - ETA: 2s - loss: 0.4945 - acc: 0.768 - ETA: 2s - loss: 0.4942 - acc: 0.768 - ETA: 2s - loss: 0.4968 - acc: 0.765 - ETA: 2s - loss: 0.4943 - acc: 0.768 - ETA: 2s - loss: 0.4941 - acc: 0.768 - ETA: 2s - loss: 0.4937 - acc: 0.768 - ETA: 2s - loss: 0.4933 - acc: 0.769 - ETA: 2s - loss: 0.4948 - acc: 0.768 - ETA: 2s - loss: 0.4943 - acc: 0.769 - ETA: 2s - loss: 0.4949 - acc: 0.769 - ETA: 2s - loss: 0.4946 - acc: 0.770 - ETA: 2s - loss: 0.4954 - acc: 0.769 - ETA: 2s - loss: 0.4943 - acc: 0.769 - ETA: 2s - loss: 0.4948 - acc: 0.769 - ETA: 2s - loss: 0.4947 - acc: 0.768 - ETA: 1s - loss: 0.4938 - acc: 0.769 - ETA: 1s - loss: 0.4928 - acc: 0.770 - ETA: 1s - loss: 0.4930 - acc: 0.769 - ETA: 1s - loss: 0.4940 - acc: 0.769 - ETA

b) add copies of instances from the under-represented class - over-sampling 

In [3]:
# create 2 dataframe with all clicked rows and select the same number of rows as in '1' from '0' data 
data_1 = data[data['Is_lp_click'] == 1]
data_1_add = data[data['Is_lp_click'] == 1].sample(15000, random_state = 32)
data_0 = data[data['Is_lp_click'] == 0].sample(50000, random_state = 12)
# concat 2 dataframes and shuffle it
data_concat = pd.concat([data_1, data_1_add, data_0])
data_under = data_concat.sample(frac=1)

In [5]:
# features & labels
labels = data_under['Is_lp_click']
features = data_under.drop('Is_lp_click', axis = 1)
# create the dictionary of unique attributes' lists
unique_attr = {}
for attr in features.columns:
    unique_attr[attr] = features[attr].unique().tolist()
unique_attr
# features encoding
encoder = preprocessing.OneHotEncoder(categories=[unique_attr[i] for i in unique_attr], sparse = False, handle_unknown='ignore')
encoder
features['CONCAT'] = features.values.tolist()
features['CONCAT'].head()
t0 = time()
features['ENCODED'] = [encoder.fit_transform([i]).flatten() for i in features['CONCAT']]
print ("time on encoding:", round(time()-t0, 3), "s")
# create the array with feature vectors
features_list = [list(i) for i in features['ENCODED']]
features_list_array = np.array(features_list)
# create the array with label vector 
labels_list_array = np.array(labels.tolist())

time on encoding: 104.521 s


In [6]:
X_train, X_test, y_train, y_test = train_test_split(features_list_array, labels_list_array, random_state=35, test_size=0.3)

In [7]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.78      0.69      0.73     15014
         1.0       0.72      0.81      0.76     15004

   micro avg       0.75      0.75      0.75     30018
   macro avg       0.75      0.75      0.75     30018
weighted avg       0.75      0.75      0.75     30018

[[10350  4664]
 [ 2894 12110]]
Accuracy is  74.82177360250516
Time on model's work: 20.968 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.62      0.73     15014
         1.0       0.71      0.92      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.80      0.77      0.76     30018
weighted avg       0.80      0.77      0.76     30018

[[ 9306  5708]
 [ 1217 13787]]
Accuracy is  76.93050836164967
Time on model's work: 1198.169 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.76      0.71      0.73     15014
         1.0       0.73      0.77      0.75     15004

   micro avg       0.74      0.74      0.74     30018
   macro avg       0.74      0.74      0.74     30018
weighted avg       0.74      0.74      0.74     30018

[[10698  4316]
 [ 3405 11599]]
Accuracy is  74.27876607368911
Time on model's work: 45.195 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.63      0.73     15014
         1.0       0.71      0.91      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.79      0.77      0.76     30018
weighted avg       0.79      0.77      0.76     30018

[[ 9384  5630]
 [ 1338 13666]]
Accuracy is  76.78726097674729
Time on model's work: 462.168 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.79      0.69      0.73   

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.87      0.63      0.73     15014
         1.0       0.71      0.90      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.79      0.77      0.76     30018
weighted avg       0.79      0.77      0.76     30018

[[ 9529  5485]
 [ 1478 13526]]
Accuracy is  76.80391764941035
Time on model's work: 3.672 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.62      0.73     15014
         1.0       0.71      0.92      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.79      0.77      0.76     30018
weighted avg       0.79      0.77      0.76     30018

[[ 9337  5677]
 [ 1265 13739]]
Accuracy is  76.87387567459524
Time on model's work: 2.009 s


MemoryError: 

In [8]:
# TFFM sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)

In [9]:
# input_type='sparse' /// rank == 10
for order in [2, 3]:
    model = TFFMClassifier(
        order=order, 
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=512,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:33<00:00,  1.51epoch/s]


[order=2] accuracy: 0.7661736291558399
[[ 9576  5438]
 [ 1581 13423]]
              precision    recall  f1-score   support

         0.0       0.86      0.64      0.73     15014
         1.0       0.71      0.89      0.79     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.78      0.77      0.76     30018
weighted avg       0.79      0.77      0.76     30018



100%|██████████| 50/50 [01:08<00:00,  1.31s/epoch]


[order=3] accuracy: 0.7636085015657272
[[ 9817  5197]
 [ 1899 13105]]
              precision    recall  f1-score   support

         0.0       0.84      0.65      0.73     15014
         1.0       0.72      0.87      0.79     15004

   micro avg       0.76      0.76      0.76     30018
   macro avg       0.78      0.76      0.76     30018
weighted avg       0.78      0.76      0.76     30018



In [10]:
# FtrlOptimizer
# input_type='sparse' /// rank == 10
model = TFFMClassifier(
    order=2, 
    rank=10, 
    optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
    n_epochs=50, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()

100%|██████████| 50/50 [00:30<00:00,  1.67epoch/s]


[order=3] accuracy: 0.7656073022852955
[[ 9270  5744]
 [ 1292 13712]]
              precision    recall  f1-score   support

         0.0       0.88      0.62      0.72     15014
         1.0       0.70      0.91      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.79      0.77      0.76     30018
weighted avg       0.79      0.77      0.76     30018



In [11]:
# sample_weight='balanced'
model = TFFMClassifier(
    order=2,
    sample_weight='balanced',
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=50, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()

100%|██████████| 50/50 [00:29<00:00,  1.69epoch/s]


[order=3] accuracy: 0.765240855486708
[[ 9499  5515]
 [ 1532 13472]]
              precision    recall  f1-score   support

         0.0       0.86      0.63      0.73     15014
         1.0       0.71      0.90      0.79     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.79      0.77      0.76     30018
weighted avg       0.79      0.77      0.76     30018



In [12]:
# weight - optional
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=20, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 20/20 [00:12<00:00,  1.67epoch/s]


[order=3] accuracy: 0.7667399560263841
[[ 9483  5531]
 [ 1471 13533]]
              precision    recall  f1-score   support

         0.0       0.87      0.63      0.73     15014
         1.0       0.71      0.90      0.79     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.79      0.77      0.76     30018
weighted avg       0.79      0.77      0.76     30018



100%|██████████| 20/20 [00:12<00:00,  1.65epoch/s]


[order=3] accuracy: 0.7668065827170365
[[ 9148  5866]
 [ 1134 13870]]
              precision    recall  f1-score   support

         0.0       0.89      0.61      0.72     15014
         1.0       0.70      0.92      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.80      0.77      0.76     30018
weighted avg       0.80      0.77      0.76     30018



100%|██████████| 20/20 [00:12<00:00,  1.68epoch/s]


[order=3] accuracy: 0.7590112599107203
[[ 8781  6233]
 [ 1001 14003]]
              precision    recall  f1-score   support

         0.0       0.90      0.58      0.71     15014
         1.0       0.69      0.93      0.79     15004

   micro avg       0.76      0.76      0.76     30018
   macro avg       0.79      0.76      0.75     30018
weighted avg       0.79      0.76      0.75     30018



100%|██████████| 20/20 [00:12<00:00,  1.67epoch/s]


[order=3] accuracy: 0.7485508694783131
[[ 8341  6673]
 [  875 14129]]
              precision    recall  f1-score   support

         0.0       0.91      0.56      0.69     15014
         1.0       0.68      0.94      0.79     15004

   micro avg       0.75      0.75      0.75     30018
   macro avg       0.79      0.75      0.74     30018
weighted avg       0.79      0.75      0.74     30018



In [13]:
# weight - 2.0 best
model = TFFMClassifier(
    order=2,
    pos_class_weight=2.0,
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=100, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()

100%|██████████| 100/100 [01:01<00:00,  1.68epoch/s]


[order=3] accuracy: 0.7647411553068159
[[ 9134  5880]
 [ 1182 13822]]
              precision    recall  f1-score   support

         0.0       0.89      0.61      0.72     15014
         1.0       0.70      0.92      0.80     15004

   micro avg       0.76      0.76      0.76     30018
   macro avg       0.79      0.76      0.76     30018
weighted avg       0.79      0.76      0.76     30018



In [14]:
# BEST OPTIMIZED TFFM
model = TFFMClassifier(
    order=2, 
    rank=10,
    pos_class_weight=2.0,
    optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
    n_epochs=100, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()

100%|██████████| 100/100 [01:00<00:00,  1.65epoch/s]


[order=3] accuracy: 0.7683056832567127
[[ 9243  5771]
 [ 1184 13820]]
              precision    recall  f1-score   support

         0.0       0.89      0.62      0.73     15014
         1.0       0.71      0.92      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.80      0.77      0.76     30018
weighted avg       0.80      0.77      0.76     30018



In [16]:
# KERAS
model = Sequential()
model.add(Dense(128, input_dim=2626, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=256)
score = model.evaluate(X_test, y_test, batch_size=256)
print(score)

Epoch 1/20
70041/70041 [==============================] - ETA: 1:30 - loss: 1.0082 - acc: 0.543 - ETA: 27s - loss: 0.8918 - acc: 0.527 - ETA: 17s - loss: 0.8464 - acc: 0.50 - ETA: 13s - loss: 0.8150 - acc: 0.50 - ETA: 11s - loss: 0.7941 - acc: 0.51 - ETA: 10s - loss: 0.7803 - acc: 0.51 - ETA: 9s - loss: 0.7692 - acc: 0.5175 - ETA: 8s - loss: 0.7596 - acc: 0.522 - ETA: 8s - loss: 0.7533 - acc: 0.527 - ETA: 7s - loss: 0.7443 - acc: 0.534 - ETA: 7s - loss: 0.7385 - acc: 0.537 - ETA: 7s - loss: 0.7349 - acc: 0.537 - ETA: 6s - loss: 0.7301 - acc: 0.541 - ETA: 6s - loss: 0.7242 - acc: 0.547 - ETA: 6s - loss: 0.7200 - acc: 0.549 - ETA: 6s - loss: 0.7152 - acc: 0.554 - ETA: 6s - loss: 0.7113 - acc: 0.558 - ETA: 5s - loss: 0.7077 - acc: 0.562 - ETA: 5s - loss: 0.7056 - acc: 0.563 - ETA: 5s - loss: 0.7022 - acc: 0.568 - ETA: 5s - loss: 0.6993 - acc: 0.571 - ETA: 5s - loss: 0.6963 - acc: 0.575 - ETA: 5s - loss: 0.6929 - acc: 0.579 - ETA: 5s - loss: 0.6906 - acc: 0.582 - ETA: 5s - loss: 0.6873 - a

70041/70041 [==============================] - ETA: 4s - loss: 0.4471 - acc: 0.824 - ETA: 4s - loss: 0.4801 - acc: 0.787 - ETA: 4s - loss: 0.4874 - acc: 0.785 - ETA: 4s - loss: 0.4974 - acc: 0.775 - ETA: 5s - loss: 0.5046 - acc: 0.770 - ETA: 5s - loss: 0.5027 - acc: 0.769 - ETA: 5s - loss: 0.5022 - acc: 0.768 - ETA: 5s - loss: 0.5055 - acc: 0.766 - ETA: 5s - loss: 0.5031 - acc: 0.768 - ETA: 5s - loss: 0.5030 - acc: 0.768 - ETA: 5s - loss: 0.5040 - acc: 0.769 - ETA: 4s - loss: 0.5034 - acc: 0.770 - ETA: 4s - loss: 0.5036 - acc: 0.770 - ETA: 4s - loss: 0.5045 - acc: 0.769 - ETA: 4s - loss: 0.5048 - acc: 0.770 - ETA: 4s - loss: 0.5048 - acc: 0.769 - ETA: 4s - loss: 0.5043 - acc: 0.769 - ETA: 4s - loss: 0.5057 - acc: 0.768 - ETA: 4s - loss: 0.5075 - acc: 0.767 - ETA: 4s - loss: 0.5062 - acc: 0.768 - ETA: 4s - loss: 0.5055 - acc: 0.769 - ETA: 4s - loss: 0.5041 - acc: 0.770 - ETA: 4s - loss: 0.5036 - acc: 0.770 - ETA: 4s - loss: 0.5033 - acc: 0.771 - ETA: 4s - loss: 0.5026 - acc: 0.772 - ETA

70041/70041 [==============================] - ETA: 5s - loss: 0.4768 - acc: 0.781 - ETA: 4s - loss: 0.4951 - acc: 0.767 - ETA: 4s - loss: 0.4960 - acc: 0.767 - ETA: 4s - loss: 0.4923 - acc: 0.769 - ETA: 4s - loss: 0.4966 - acc: 0.768 - ETA: 4s - loss: 0.4972 - acc: 0.767 - ETA: 4s - loss: 0.4965 - acc: 0.769 - ETA: 4s - loss: 0.4948 - acc: 0.771 - ETA: 4s - loss: 0.4898 - acc: 0.773 - ETA: 4s - loss: 0.4898 - acc: 0.772 - ETA: 4s - loss: 0.4877 - acc: 0.774 - ETA: 4s - loss: 0.4868 - acc: 0.775 - ETA: 4s - loss: 0.4867 - acc: 0.775 - ETA: 4s - loss: 0.4865 - acc: 0.776 - ETA: 4s - loss: 0.4873 - acc: 0.776 - ETA: 4s - loss: 0.4880 - acc: 0.776 - ETA: 4s - loss: 0.4867 - acc: 0.777 - ETA: 4s - loss: 0.4855 - acc: 0.778 - ETA: 4s - loss: 0.4855 - acc: 0.778 - ETA: 3s - loss: 0.4859 - acc: 0.778 - ETA: 3s - loss: 0.4870 - acc: 0.778 - ETA: 3s - loss: 0.4885 - acc: 0.777 - ETA: 3s - loss: 0.4874 - acc: 0.778 - ETA: 3s - loss: 0.4869 - acc: 0.778 - ETA: 3s - loss: 0.4870 - acc: 0.779 - ETA

70041/70041 [==============================] - ETA: 5s - loss: 0.5220 - acc: 0.765 - ETA: 5s - loss: 0.4891 - acc: 0.786 - ETA: 4s - loss: 0.4976 - acc: 0.774 - ETA: 4s - loss: 0.4898 - acc: 0.776 - ETA: 4s - loss: 0.4891 - acc: 0.779 - ETA: 4s - loss: 0.4879 - acc: 0.779 - ETA: 4s - loss: 0.4842 - acc: 0.780 - ETA: 4s - loss: 0.4838 - acc: 0.780 - ETA: 4s - loss: 0.4842 - acc: 0.780 - ETA: 4s - loss: 0.4853 - acc: 0.777 - ETA: 4s - loss: 0.4814 - acc: 0.781 - ETA: 4s - loss: 0.4858 - acc: 0.778 - ETA: 4s - loss: 0.4875 - acc: 0.777 - ETA: 4s - loss: 0.4845 - acc: 0.778 - ETA: 4s - loss: 0.4826 - acc: 0.779 - ETA: 4s - loss: 0.4829 - acc: 0.780 - ETA: 4s - loss: 0.4826 - acc: 0.780 - ETA: 4s - loss: 0.4829 - acc: 0.780 - ETA: 4s - loss: 0.4828 - acc: 0.780 - ETA: 4s - loss: 0.4840 - acc: 0.780 - ETA: 4s - loss: 0.4851 - acc: 0.778 - ETA: 3s - loss: 0.4859 - acc: 0.777 - ETA: 3s - loss: 0.4864 - acc: 0.777 - ETA: 3s - loss: 0.4852 - acc: 0.777 - ETA: 3s - loss: 0.4863 - acc: 0.776 - ETA

70041/70041 [==============================] - ETA: 5s - loss: 0.5460 - acc: 0.730 - ETA: 4s - loss: 0.5164 - acc: 0.753 - ETA: 4s - loss: 0.5085 - acc: 0.760 - ETA: 4s - loss: 0.5044 - acc: 0.757 - ETA: 4s - loss: 0.5006 - acc: 0.760 - ETA: 4s - loss: 0.4973 - acc: 0.762 - ETA: 4s - loss: 0.4937 - acc: 0.765 - ETA: 4s - loss: 0.4929 - acc: 0.766 - ETA: 4s - loss: 0.4934 - acc: 0.767 - ETA: 4s - loss: 0.4917 - acc: 0.769 - ETA: 4s - loss: 0.4924 - acc: 0.769 - ETA: 4s - loss: 0.4915 - acc: 0.769 - ETA: 4s - loss: 0.4922 - acc: 0.768 - ETA: 4s - loss: 0.4944 - acc: 0.765 - ETA: 4s - loss: 0.4929 - acc: 0.767 - ETA: 4s - loss: 0.4929 - acc: 0.766 - ETA: 4s - loss: 0.4914 - acc: 0.767 - ETA: 4s - loss: 0.4928 - acc: 0.767 - ETA: 4s - loss: 0.4923 - acc: 0.767 - ETA: 4s - loss: 0.4908 - acc: 0.769 - ETA: 4s - loss: 0.4898 - acc: 0.770 - ETA: 4s - loss: 0.4883 - acc: 0.771 - ETA: 3s - loss: 0.4890 - acc: 0.771 - ETA: 3s - loss: 0.4899 - acc: 0.771 - ETA: 3s - loss: 0.4896 - acc: 0.771 - ETA

Best results: 
1. Undersampling
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.89      0.62      0.73      6986
         1.0       0.71      0.92      0.80      7038

   micro avg       0.77      0.77      0.77     14024
   macro avg       0.80      0.77      0.76     14024
weighted avg       0.80      0.77      0.76     14024

[[4304 2682]
 [ 537 6501]]
Accuracy is  77.04649172846548
Time on model's work: 736.97 s
============================
XGBClassifier
              precision    recall  f1-score   support

         0.0       0.89      0.61      0.73      6986
         1.0       0.71      0.93      0.80      7038

   micro avg       0.77      0.77      0.77     14024
   macro avg       0.80      0.77      0.76     14024
weighted avg       0.80      0.77      0.76     14024

[[4288 2698]
 [ 524 6514]]
Accuracy is  77.02509982886481
Time on model's work: 480.744 s
============================
TFFMClassifier
[order=2] accuracy: 0.7678265830005705
[[4326 2715]
 [ 541 6442]]
              precision    recall  f1-score   support

         0.0       0.89      0.61      0.73      7041
         1.0       0.70      0.92      0.80      6983

   micro avg       0.77      0.77      0.77     14024
   macro avg       0.80      0.77      0.76     14024
weighted avg       0.80      0.77      0.76     14024

2. Oversampling
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.62      0.73     15014
         1.0       0.71      0.92      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.80      0.77      0.76     30018
weighted avg       0.80      0.77      0.76     30018

[[ 9306  5708]
 [ 1217 13787]]
Accuracy is  76.93050836164967
Time on model's work: 1198.169 s
============================
XGBClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.62      0.73     15014
         1.0       0.71      0.92      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.80      0.77      0.76     30018
weighted avg       0.80      0.77      0.76     30018

[[ 9299  5715]
 [ 1212 13792]]
Accuracy is  76.92384569258445
Time on model's work: 561.64 s
============================
CatBoostClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.63      0.73     15014
         1.0       0.71      0.91      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.79      0.77      0.76     30018
weighted avg       0.79      0.77      0.76     30018

[[ 9406  5608]
 [ 1304 13700]]
Accuracy is  76.97381571057366
Time on model's work: 318.586 s
============================
TFFMClassifier
[order=2] accuracy: 0.7683056832567127
[[ 9243  5771]
 [ 1184 13820]]
              precision    recall  f1-score   support

         0.0       0.89      0.62      0.73     15014
         1.0       0.71      0.92      0.80     15004

   micro avg       0.77      0.77      0.77     30018
   macro avg       0.80      0.77      0.76     30018
weighted avg       0.80      0.77      0.76     30018



### Undersampling strategy shows better results than oversampling by time, accuracy, precision, recall, f1-score

 ## 2. Imbalanced variant

In [10]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.metrics import balanced_accuracy_score
t = time()
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)
bbc.fit(X_train, y_train) 
predictions = bbc.predict(X_test)
print('Balanced accuracy', balanced_accuracy_score(y_test, predictions)) 
print('Accuracy: {}'.format(accuracy_score(y_test, predictions)))
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
print ("Time on model's work:", round(time()-t, 3), "s")

Balanced accuracy 0.7042175138171767
Accuracy: 0.7163
[[6662 2612]
 [ 225  501]]
              precision    recall  f1-score   support

         0.0       0.97      0.72      0.82      9274
         1.0       0.16      0.69      0.26       726

   micro avg       0.72      0.72      0.72     10000
   macro avg       0.56      0.70      0.54     10000
weighted avg       0.91      0.72      0.78     10000

Time on model's work: 53.088 s


In [12]:
from imblearn.ensemble import BalancedRandomForestClassifier
t = time()
brf = BalancedBaggingClassifier(n_estimators=100, random_state=0)
brf.fit(X_train, y_train) 
predictions = brf.predict(X_test)
print('Balanced accuracy', balanced_accuracy_score(y_test, predictions)) 
print('Accuracy: {}'.format(accuracy_score(y_test, predictions)))
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
print ("Time on model's work:", round(time()-t, 3), "s")

Balanced accuracy 0.729588214570668
Accuracy: 0.6986
[[6430 2844]
 [ 170  556]]
              precision    recall  f1-score   support

         0.0       0.97      0.69      0.81      9274
         1.0       0.16      0.77      0.27       726

   micro avg       0.70      0.70      0.70     10000
   macro avg       0.57      0.73      0.54     10000
weighted avg       0.92      0.70      0.77     10000

Time on model's work: 496.02 s


In [13]:
from imblearn.ensemble import RUSBoostClassifier
t = time()
rusboost = RUSBoostClassifier(random_state=10)
rusboost.fit(X_train, y_train)  
predictions = rusboost.predict(X_test)
print('Balanced accuracy', balanced_accuracy_score(y_test, predictions)) 
print('Accuracy: {}'.format(accuracy_score(y_test, predictions)))
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
print ("Time on model's work:", round(time()-t, 3), "s")

Balanced accuracy 0.5633828630770227
Accuracy: 0.7824
[[7601 1673]
 [ 503  223]]
              precision    recall  f1-score   support

         0.0       0.94      0.82      0.87      9274
         1.0       0.12      0.31      0.17       726

   micro avg       0.78      0.78      0.78     10000
   macro avg       0.53      0.56      0.52     10000
weighted avg       0.88      0.78      0.82     10000

Time on model's work: 42.791 s


In [14]:
from imblearn.ensemble import EasyEnsembleClassifier
t = time()
eec = EasyEnsembleClassifier(random_state=11)
eec.fit(X_train, y_train) 
predictions = eec.predict(X_test)
print('Balanced accuracy', balanced_accuracy_score(y_test, predictions)) 
print('Accuracy: {}'.format(accuracy_score(y_test, predictions)))
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
print ("Time on model's work:", round(time()-t, 3), "s")

Balanced accuracy 0.7680685241657266
Accuracy: 0.6487
[[5828 3446]
 [  67  659]]
              precision    recall  f1-score   support

         0.0       0.99      0.63      0.77      9274
         1.0       0.16      0.91      0.27       726

   micro avg       0.65      0.65      0.65     10000
   macro avg       0.57      0.77      0.52     10000
weighted avg       0.93      0.65      0.73     10000

Time on model's work: 135.531 s


In [15]:
print(sorted(Counter(labels_list_array).items()))

[(0.0, 46269), (1.0, 3731)]


## RandomUnderSampler

In [16]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(features_list_array, labels_list_array)
print(sorted(Counter(y_resampled).items()))

[(0.0, 3731), (1.0, 3731)]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=35, test_size=0.2)

In [18]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.77      0.72      0.74       760
         1.0       0.73      0.77      0.75       733

   micro avg       0.75      0.75      0.75      1493
   macro avg       0.75      0.75      0.75      1493
weighted avg       0.75      0.75      0.75      1493

[[546 214]
 [165 568]]
Accuracy is  74.61486939048895
Time on model's work: 0.85 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.64      0.74       760
         1.0       0.71      0.91      0.79       733

   micro avg       0.77      0.77      0.77      1493
   macro avg       0.79      0.77      0.77      1493
weighted avg       0.79      0.77      0.77      1493

[[484 276]
 [ 69 664]]
Accuracy is  76.8921634293369
Time on model's work: 36.23 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.75      0.72      0.73       760
         1.0       0.72      0.74      0.73       733

   micro avg       0.73      0.73      0.73      1493
   macro avg       0.73      0.73      0.73      1493
weighted avg       0.73      0.73      0.73      1493

[[550 210]
 [188 545]]
Accuracy is  73.34226389819156
Time on model's work: 1.31 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.86      0.66      0.75       760
         1.0       0.72      0.89      0.79       733

   micro avg       0.77      0.77      0.77      1493
   macro avg       0.79      0.78      0.77      1493
weighted avg       0.79      0.77      0.77      1493

[[504 256]
 [ 82 651]]
Accuracy is  77.36101808439383
Time on model's work: 8.838 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.75      0.71      0.73       760
         1.0

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

936:	learn: 0.4601396	total: 57.2s	remaining: 3.85s
937:	learn: 0.4601085	total: 57.3s	remaining: 3.79s
938:	learn: 0.4600601	total: 57.4s	remaining: 3.73s
939:	learn: 0.4600427	total: 57.4s	remaining: 3.66s
940:	learn: 0.4600327	total: 57.5s	remaining: 3.6s
941:	learn: 0.4599979	total: 57.5s	remaining: 3.54s
942:	learn: 0.4599841	total: 57.6s	remaining: 3.48s
943:	learn: 0.4599754	total: 57.6s	remaining: 3.42s
944:	learn: 0.4599459	total: 57.7s	remaining: 3.36s
945:	learn: 0.4599291	total: 57.8s	remaining: 3.3s
946:	learn: 0.4598938	total: 57.8s	remaining: 3.23s
947:	learn: 0.4598587	total: 57.9s	remaining: 3.17s
948:	learn: 0.4597615	total: 57.9s	remaining: 3.11s
949:	learn: 0.4597471	total: 58s	remaining: 3.05s
950:	learn: 0.4597312	total: 58s	remaining: 2.99s
951:	learn: 0.4597102	total: 58.1s	remaining: 2.93s
952:	learn: 0.4596861	total: 58.2s	remaining: 2.87s
953:	learn: 0.4596746	total: 58.2s	remaining: 2.81s
954:	learn: 0.4596570	total: 58.3s	remaining: 2.75s
955:	learn: 0.4596

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.86      0.66      0.75       760
         1.0       0.72      0.89      0.79       733

   micro avg       0.77      0.77      0.77      1493
   macro avg       0.79      0.78      0.77      1493
weighted avg       0.79      0.77      0.77      1493

[[504 256]
 [ 81 652]]
Accuracy is  77.42799732083054
Time on model's work: 0.144 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.71      0.65      0.68       760
         1.0       0.67      0.73      0.70       733

   micro avg       0.69      0.69      0.69      1493
   macro avg       0.69      0.69      0.69      1493
weighted avg       0.69      0.69      0.69      1493

[[492 268]
 [197 536]]
Accuracy is  68.85465505693234
Time on model's work: 0.182 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.79      0.68      0.73       760
         1.0       0.71      0.81      0.76       733

   micro avg       0.75      0.75      0.75      1493
   macro avg       0.75      0.75      0.74      1493
weighted avg       0.75      0.75      0.74      1493

[[519 241]
 [139 594]]
Accuracy is  74.54789015405224
Time on model's work: 18.646 s
TOTAL TIME SPENT:  296.608 s


In [22]:
# TFFM sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)
# BEST OPTIMIZED TFFM
model = TFFMClassifier(
    order=2, 
    rank=10,
    pos_class_weight=2.0,
    optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
    n_epochs=100, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    #log_dir='./tmp/logs',
    #verbose=1,
    seed=42
)
model.fit(X_train_sparse, y_train, show_progress=True)
predictions = model.predict(X_test_sparse)
print('Accuracy: {}'.format(accuracy_score(y_test, predictions)))
# this will close tf.Session and free resources
print(confusion_matrix(y_test,predictions)) 
print(classification_report(y_test, predictions))
model.destroy()

100%|██████████| 100/100 [00:05<00:00, 19.61epoch/s]


Accuracy: 0.7387809778968519
[[456 304]
 [ 86 647]]
              precision    recall  f1-score   support

         0.0       0.84      0.60      0.70       760
         1.0       0.68      0.88      0.77       733

   micro avg       0.74      0.74      0.74      1493
   macro avg       0.76      0.74      0.73      1493
weighted avg       0.76      0.74      0.73      1493



## NearMiss (version = 1)

In [4]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(version=1)
X_resampled_nm1, y_resampled1 = nm1.fit_resample(features_list_array, labels_list_array)
print(sorted(Counter(y_resampled1).items()))

[(0.0, 3726), (1.0, 3726)]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled_nm1, y_resampled1, random_state=35, test_size=0.2)

In [6]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.83      0.94      0.88       763
         1.0       0.92      0.79      0.85       728

   micro avg       0.87      0.87      0.87      1491
   macro avg       0.87      0.86      0.87      1491
weighted avg       0.87      0.87      0.87      1491

[[714  49]
 [150 578]]
Accuracy is  86.65325285043595
Time on model's work: 0.767 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87       763
         1.0       0.86      0.86      0.86       728

   micro avg       0.86      0.86      0.86      1491
   macro avg       0.86      0.86      0.86      1491
weighted avg       0.86      0.86      0.86      1491

[[661 102]
 [100 628]]
Accuracy is  86.45204560697518
Time on model's work: 31.618 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.82      0.96      0.88       763
         1.0       0.94      0.78      0.85       728

   micro avg       0.87      0.87      0.87      1491
   macro avg       0.88      0.87      0.87      1491
weighted avg       0.88      0.87      0.87      1491

[[730  33]
 [163 565]]
Accuracy is  86.85446009389672
Time on model's work: 1.506 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.86      0.86      0.86       763
         1.0       0.85      0.85      0.85       728

   micro avg       0.86      0.86      0.86      1491
   macro avg       0.86      0.86      0.86      1491
weighted avg       0.86      0.86      0.86      1491

[[656 107]
 [108 620]]
Accuracy is  85.58014755197854
Time on model's work: 9.615 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.82      0.94      0.87       763
         1.

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

936:	learn: 0.2805619	total: 58.4s	remaining: 3.93s
937:	learn: 0.2805608	total: 58.5s	remaining: 3.86s
938:	learn: 0.2805528	total: 58.5s	remaining: 3.8s
939:	learn: 0.2805505	total: 58.6s	remaining: 3.74s
940:	learn: 0.2805488	total: 58.7s	remaining: 3.68s
941:	learn: 0.2805435	total: 58.7s	remaining: 3.62s
942:	learn: 0.2804272	total: 58.8s	remaining: 3.55s
943:	learn: 0.2804168	total: 58.9s	remaining: 3.49s
944:	learn: 0.2804098	total: 58.9s	remaining: 3.43s
945:	learn: 0.2804013	total: 59s	remaining: 3.37s
946:	learn: 0.2803941	total: 59.1s	remaining: 3.31s
947:	learn: 0.2803897	total: 59.1s	remaining: 3.24s
948:	learn: 0.2803713	total: 59.2s	remaining: 3.18s
949:	learn: 0.2803705	total: 59.3s	remaining: 3.12s
950:	learn: 0.2803648	total: 59.3s	remaining: 3.06s
951:	learn: 0.2803581	total: 59.4s	remaining: 2.99s
952:	learn: 0.2803505	total: 59.5s	remaining: 2.93s
953:	learn: 0.2803242	total: 59.6s	remaining: 2.87s
954:	learn: 0.2802738	total: 59.6s	remaining: 2.81s
955:	learn: 0.2

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.86      0.89      0.87       763
         1.0       0.88      0.85      0.86       728

   micro avg       0.87      0.87      0.87      1491
   macro avg       0.87      0.87      0.87      1491
weighted avg       0.87      0.87      0.87      1491

[[676  87]
 [111 617]]
Accuracy is  86.72032193158954
Time on model's work: 0.131 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.58      0.70       763
         1.0       0.68      0.92      0.78       728

   micro avg       0.75      0.75      0.75      1491
   macro avg       0.78      0.75      0.74      1491
weighted avg       0.78      0.75      0.74      1491

[[442 321]
 [ 59 669]]
Accuracy is  74.51374916163648
Time on model's work: 0.159 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.84      0.91      0.88       763
         1.0       0.90      0.82      0.86       728

   micro avg       0.87      0.87      0.87      1491
   macro avg       0.87      0.87      0.87      1491
weighted avg       0.87      0.87      0.87      1491

[[695  68]
 [130 598]]
Accuracy is  86.72032193158954
Time on model's work: 19.6 s
TOTAL TIME SPENT:  298.635 s


In [8]:
# TFFM sparse - works worse with sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)
# weight - optional / AdamOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=20, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 20/20 [00:01<00:00, 17.37epoch/s]


accuracy: 0.8443997317236754
[[649 114]
 [118 610]]
              precision    recall  f1-score   support

         0.0       0.85      0.85      0.85       763
         1.0       0.84      0.84      0.84       728

   micro avg       0.84      0.84      0.84      1491
   macro avg       0.84      0.84      0.84      1491
weighted avg       0.84      0.84      0.84      1491



100%|██████████| 20/20 [00:01<00:00, 16.57epoch/s]


accuracy: 0.7665995975855131
[[458 305]
 [ 43 685]]
              precision    recall  f1-score   support

         0.0       0.91      0.60      0.72       763
         1.0       0.69      0.94      0.80       728

   micro avg       0.77      0.77      0.77      1491
   macro avg       0.80      0.77      0.76      1491
weighted avg       0.81      0.77      0.76      1491



100%|██████████| 20/20 [00:01<00:00, 16.73epoch/s]


accuracy: 0.6955063715627096
[[323 440]
 [ 14 714]]
              precision    recall  f1-score   support

         0.0       0.96      0.42      0.59       763
         1.0       0.62      0.98      0.76       728

   micro avg       0.70      0.70      0.70      1491
   macro avg       0.79      0.70      0.67      1491
weighted avg       0.79      0.70      0.67      1491



100%|██████████| 20/20 [00:01<00:00, 15.72epoch/s]


accuracy: 0.7015425888665325
[[330 433]
 [ 12 716]]
              precision    recall  f1-score   support

         0.0       0.96      0.43      0.60       763
         1.0       0.62      0.98      0.76       728

   micro avg       0.70      0.70      0.70      1491
   macro avg       0.79      0.71      0.68      1491
weighted avg       0.80      0.70      0.68      1491



In [9]:
# weight - optional / FtrlOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
        n_epochs=20, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 20/20 [00:01<00:00, 16.32epoch/s]


accuracy: 0.7002012072434608
[[384 379]
 [ 68 660]]
              precision    recall  f1-score   support

         0.0       0.85      0.50      0.63       763
         1.0       0.64      0.91      0.75       728

   micro avg       0.70      0.70      0.70      1491
   macro avg       0.74      0.70      0.69      1491
weighted avg       0.74      0.70      0.69      1491



100%|██████████| 20/20 [00:01<00:00, 16.28epoch/s]


accuracy: 0.7136150234741784
[[366 397]
 [ 30 698]]
              precision    recall  f1-score   support

         0.0       0.92      0.48      0.63       763
         1.0       0.64      0.96      0.77       728

   micro avg       0.71      0.71      0.71      1491
   macro avg       0.78      0.72      0.70      1491
weighted avg       0.78      0.71      0.70      1491



100%|██████████| 20/20 [00:01<00:00, 16.19epoch/s]


accuracy: 0.6747149564050973
[[292 471]
 [ 14 714]]
              precision    recall  f1-score   support

         0.0       0.95      0.38      0.55       763
         1.0       0.60      0.98      0.75       728

   micro avg       0.67      0.67      0.67      1491
   macro avg       0.78      0.68      0.65      1491
weighted avg       0.78      0.67      0.64      1491



100%|██████████| 20/20 [00:01<00:00, 16.94epoch/s]


accuracy: 0.6753856472166331
[[292 471]
 [ 13 715]]
              precision    recall  f1-score   support

         0.0       0.96      0.38      0.55       763
         1.0       0.60      0.98      0.75       728

   micro avg       0.68      0.68      0.68      1491
   macro avg       0.78      0.68      0.65      1491
weighted avg       0.78      0.68      0.64      1491



In [14]:
# KERAS
X_train.shape

(5961, 2226)

In [15]:
model = Sequential()
model.add(Dense(128, input_dim=2226, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=256)
score = model.evaluate(X_test, y_test, batch_size=256)
print(score)

Epoch 1/20
5961/5961 [==============================] - ETA: 6s - loss: 0.9735 - acc: 0.515 - ETA: 1s - loss: 0.8441 - acc: 0.496 - ETA: 0s - loss: 0.8211 - acc: 0.502 - ETA: 0s - loss: 0.8052 - acc: 0.495 - ETA: 0s - loss: 0.7865 - acc: 0.503 - ETA: 0s - loss: 0.7811 - acc: 0.502 - ETA: 0s - loss: 0.7739 - acc: 0.508 - ETA: 0s - loss: 0.7696 - acc: 0.510 - 1s 129us/step - loss: 0.7651 - acc: 0.5108
Epoch 2/20
5961/5961 [==============================] - ETA: 0s - loss: 0.7403 - acc: 0.488 - ETA: 0s - loss: 0.7087 - acc: 0.541 - ETA: 0s - loss: 0.6961 - acc: 0.563 - ETA: 0s - loss: 0.6881 - acc: 0.575 - ETA: 0s - loss: 0.6828 - acc: 0.582 - ETA: 0s - loss: 0.6814 - acc: 0.583 - ETA: 0s - loss: 0.6772 - acc: 0.585 - ETA: 0s - loss: 0.6768 - acc: 0.586 - ETA: 0s - loss: 0.6720 - acc: 0.590 - 1s 87us/step - loss: 0.6726 - acc: 0.5895
Epoch 3/20
5961/5961 [==============================] - ETA: 0s - loss: 0.6494 - acc: 0.621 - ETA: 0s - loss: 0.6422 - acc: 0.628 - ETA: 0s - loss: 0.6352 - 

## NearMiss (version = 2)

In [16]:
nm2 = NearMiss(version=2)
X_resampled_nm2, y_resampled2 = nm2.fit_resample(features_list_array, labels_list_array)
print(sorted(Counter(y_resampled2).items()))

[(0.0, 3726), (1.0, 3726)]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled_nm2, y_resampled2, random_state=35, test_size=0.2)

In [18]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.80      0.82      0.81       763
         1.0       0.81      0.78      0.79       728

   micro avg       0.80      0.80      0.80      1491
   macro avg       0.80      0.80      0.80      1491
weighted avg       0.80      0.80      0.80      1491

[[625 138]
 [157 571]]
Accuracy is  80.21462105969148
Time on model's work: 0.722 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.79      0.88      0.83       763
         1.0       0.86      0.75      0.80       728

   micro avg       0.82      0.82      0.82      1491
   macro avg       0.82      0.82      0.82      1491
weighted avg       0.82      0.82      0.82      1491

[[671  92]
 [181 547]]
Accuracy is  81.69014084507043
Time on model's work: 36.396 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.80      0.84      0.82       763
         1.0       0.83      0.78      0.80       728

   micro avg       0.81      0.81      0.81      1491
   macro avg       0.81      0.81      0.81      1491
weighted avg       0.81      0.81      0.81      1491

[[644 119]
 [160 568]]
Accuracy is  81.28772635814889
Time on model's work: 1.25 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.86      0.72      0.79       763
         1.0       0.75      0.88      0.81       728

   micro avg       0.80      0.80      0.80      1491
   macro avg       0.80      0.80      0.80      1491
weighted avg       0.81      0.80      0.80      1491

[[552 211]
 [ 91 637]]
Accuracy is  79.74513749161638
Time on model's work: 9.626 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.79      0.87      0.83       763
         1.0

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

937:	learn: 0.2793619	total: 1m	remaining: 4.03s
938:	learn: 0.2792465	total: 1m 1s	remaining: 3.96s
939:	learn: 0.2792380	total: 1m 1s	remaining: 3.9s
940:	learn: 0.2792269	total: 1m 1s	remaining: 3.83s
941:	learn: 0.2792213	total: 1m 1s	remaining: 3.77s
942:	learn: 0.2792029	total: 1m 1s	remaining: 3.71s
943:	learn: 0.2791362	total: 1m 1s	remaining: 3.64s
944:	learn: 0.2791292	total: 1m 1s	remaining: 3.58s
945:	learn: 0.2791112	total: 1m 1s	remaining: 3.51s
946:	learn: 0.2791054	total: 1m 1s	remaining: 3.45s
947:	learn: 0.2790948	total: 1m 1s	remaining: 3.38s
948:	learn: 0.2790210	total: 1m 1s	remaining: 3.32s
949:	learn: 0.2790054	total: 1m 1s	remaining: 3.25s
950:	learn: 0.2789956	total: 1m 1s	remaining: 3.19s
951:	learn: 0.2789875	total: 1m 1s	remaining: 3.12s
952:	learn: 0.2789685	total: 1m 2s	remaining: 3.06s
953:	learn: 0.2789524	total: 1m 2s	remaining: 2.99s
954:	learn: 0.2789328	total: 1m 2s	remaining: 2.93s
955:	learn: 0.2789261	total: 1m 2s	remaining: 2.86s
956:	learn: 0.27

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.84      0.80      0.82       763
         1.0       0.80      0.84      0.82       728

   micro avg       0.82      0.82      0.82      1491
   macro avg       0.82      0.82      0.82      1491
weighted avg       0.82      0.82      0.82      1491

[[608 155]
 [117 611]]
Accuracy is  81.75720992622401
Time on model's work: 0.127 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.92      0.58      0.72       763
         1.0       0.69      0.95      0.80       728

   micro avg       0.76      0.76      0.76      1491
   macro avg       0.80      0.77      0.76      1491
weighted avg       0.81      0.76      0.75      1491

[[446 317]
 [ 38 690]]
Accuracy is  76.19047619047619
Time on model's work: 0.172 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.83      0.85      0.84       763
         1.0       0.83      0.81      0.82       728

   micro avg       0.83      0.83      0.83      1491
   macro avg       0.83      0.83      0.83      1491
weighted avg       0.83      0.83      0.83      1491

[[645 118]
 [136 592]]
Accuracy is  82.9644533869886
Time on model's work: 20.84 s
TOTAL TIME SPENT:  314.363 s


In [22]:
# TFFM sparse - works worse with sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)
# weight - optional / AdamOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:02<00:00, 18.27epoch/s]


accuracy: 0.8356807511737089
[[658 105]
 [140 588]]
              precision    recall  f1-score   support

         0.0       0.82      0.86      0.84       763
         1.0       0.85      0.81      0.83       728

   micro avg       0.84      0.84      0.84      1491
   macro avg       0.84      0.84      0.84      1491
weighted avg       0.84      0.84      0.84      1491



100%|██████████| 50/50 [00:02<00:00, 17.32epoch/s]


accuracy: 0.7994634473507712
[[535 228]
 [ 71 657]]
              precision    recall  f1-score   support

         0.0       0.88      0.70      0.78       763
         1.0       0.74      0.90      0.81       728

   micro avg       0.80      0.80      0.80      1491
   macro avg       0.81      0.80      0.80      1491
weighted avg       0.81      0.80      0.80      1491



100%|██████████| 50/50 [00:02<00:00, 19.44epoch/s]


accuracy: 0.784037558685446
[[491 272]
 [ 50 678]]
              precision    recall  f1-score   support

         0.0       0.91      0.64      0.75       763
         1.0       0.71      0.93      0.81       728

   micro avg       0.78      0.78      0.78      1491
   macro avg       0.81      0.79      0.78      1491
weighted avg       0.81      0.78      0.78      1491



100%|██████████| 50/50 [00:02<00:00, 20.03epoch/s]


accuracy: 0.7739771965124078
[[460 303]
 [ 34 694]]
              precision    recall  f1-score   support

         0.0       0.93      0.60      0.73       763
         1.0       0.70      0.95      0.80       728

   micro avg       0.77      0.77      0.77      1491
   macro avg       0.81      0.78      0.77      1491
weighted avg       0.82      0.77      0.77      1491



In [23]:
# weight - optional / FtrlOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:02<00:00, 17.57epoch/s]


accuracy: 0.7323943661971831
[[456 307]
 [ 92 636]]
              precision    recall  f1-score   support

         0.0       0.83      0.60      0.70       763
         1.0       0.67      0.87      0.76       728

   micro avg       0.73      0.73      0.73      1491
   macro avg       0.75      0.74      0.73      1491
weighted avg       0.76      0.73      0.73      1491



100%|██████████| 50/50 [00:02<00:00, 18.23epoch/s]


accuracy: 0.7377598926894702
[[445 318]
 [ 73 655]]
              precision    recall  f1-score   support

         0.0       0.86      0.58      0.69       763
         1.0       0.67      0.90      0.77       728

   micro avg       0.74      0.74      0.74      1491
   macro avg       0.77      0.74      0.73      1491
weighted avg       0.77      0.74      0.73      1491



100%|██████████| 50/50 [00:02<00:00, 18.37epoch/s]


accuracy: 0.7431254191817572
[[444 319]
 [ 64 664]]
              precision    recall  f1-score   support

         0.0       0.87      0.58      0.70       763
         1.0       0.68      0.91      0.78       728

   micro avg       0.74      0.74      0.74      1491
   macro avg       0.77      0.75      0.74      1491
weighted avg       0.78      0.74      0.74      1491



100%|██████████| 50/50 [00:02<00:00, 18.25epoch/s]


accuracy: 0.7592219986586184
[[436 327]
 [ 32 696]]
              precision    recall  f1-score   support

         0.0       0.93      0.57      0.71       763
         1.0       0.68      0.96      0.79       728

   micro avg       0.76      0.76      0.76      1491
   macro avg       0.81      0.76      0.75      1491
weighted avg       0.81      0.76      0.75      1491



In [27]:
# KERAS
X_train.shape[1]

2226

In [28]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=256)
score = model.evaluate(X_test, y_test, batch_size=256)
print(score)

Epoch 1/20
5961/5961 [==============================] - ETA: 7s - loss: 0.7537 - acc: 0.558 - ETA: 1s - loss: 0.7367 - acc: 0.515 - ETA: 1s - loss: 0.7329 - acc: 0.522 - ETA: 0s - loss: 0.7321 - acc: 0.519 - ETA: 0s - loss: 0.7309 - acc: 0.518 - ETA: 0s - loss: 0.7275 - acc: 0.518 - ETA: 0s - loss: 0.7217 - acc: 0.521 - ETA: 0s - loss: 0.7211 - acc: 0.522 - 1s 130us/step - loss: 0.7193 - acc: 0.5251
Epoch 2/20
5961/5961 [==============================] - ETA: 0s - loss: 0.7055 - acc: 0.535 - ETA: 0s - loss: 0.6957 - acc: 0.539 - ETA: 0s - loss: 0.6938 - acc: 0.555 - ETA: 0s - loss: 0.6900 - acc: 0.557 - ETA: 0s - loss: 0.6884 - acc: 0.555 - ETA: 0s - loss: 0.6833 - acc: 0.564 - ETA: 0s - loss: 0.6818 - acc: 0.567 - ETA: 0s - loss: 0.6802 - acc: 0.570 - 0s 79us/step - loss: 0.6789 - acc: 0.5715
Epoch 3/20
5961/5961 [==============================] - ETA: 0s - loss: 0.6451 - acc: 0.675 - ETA: 0s - loss: 0.6532 - acc: 0.642 - ETA: 0s - loss: 0.6517 - acc: 0.631 - ETA: 0s - loss: 0.6497 - 

## NearMiss (version = 3)

In [29]:
nm3 = NearMiss(version=3)
X_resampled_nm3, y_resampled3 = nm3.fit_resample(features_list_array, labels_list_array)
print(sorted(Counter(y_resampled2).items()))

[(0.0, 3726), (1.0, 3726)]


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled_nm3, y_resampled3, random_state=35, test_size=0.2)

In [31]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.61      0.69      0.65       763
         1.0       0.62      0.55      0.58       728

   micro avg       0.62      0.62      0.62      1491
   macro avg       0.62      0.62      0.62      1491
weighted avg       0.62      0.62      0.62      1491

[[524 239]
 [330 398]]
Accuracy is  61.837692823608315
Time on model's work: 0.907 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.62      0.74      0.67       763
         1.0       0.66      0.53      0.59       728

   micro avg       0.64      0.64      0.64      1491
   macro avg       0.64      0.63      0.63      1491
weighted avg       0.64      0.64      0.63      1491

[[562 201]
 [343 385]]
Accuracy is  63.51441985244802
Time on model's work: 35.264 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.59      0.66      0.62       763
         1.0       0.59      0.51      0.55       728

   micro avg       0.59      0.59      0.59      1491
   macro avg       0.59      0.58      0.58      1491
weighted avg       0.59      0.59      0.58      1491

[[500 263]
 [354 374]]
Accuracy is  58.61837692823608
Time on model's work: 1.553 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.62      0.70      0.66       763
         1.0       0.64      0.56      0.59       728

   micro avg       0.63      0.63      0.63      1491
   macro avg       0.63      0.63      0.63      1491
weighted avg       0.63      0.63      0.63      1491

[[532 231]
 [322 406]]
Accuracy is  62.91079812206573
Time on model's work: 9.505 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.62      0.75      0.68       763
         1.

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

934:	learn: 0.5678095	total: 1m 6s	remaining: 4.63s
935:	learn: 0.5677453	total: 1m 6s	remaining: 4.56s
936:	learn: 0.5676420	total: 1m 6s	remaining: 4.49s
937:	learn: 0.5676173	total: 1m 6s	remaining: 4.42s
938:	learn: 0.5676035	total: 1m 6s	remaining: 4.35s
939:	learn: 0.5675919	total: 1m 6s	remaining: 4.27s
940:	learn: 0.5675682	total: 1m 7s	remaining: 4.2s
941:	learn: 0.5675423	total: 1m 7s	remaining: 4.13s
942:	learn: 0.5674986	total: 1m 7s	remaining: 4.06s
943:	learn: 0.5674316	total: 1m 7s	remaining: 3.99s
944:	learn: 0.5674041	total: 1m 7s	remaining: 3.92s
945:	learn: 0.5673762	total: 1m 7s	remaining: 3.85s
946:	learn: 0.5673639	total: 1m 7s	remaining: 3.77s
947:	learn: 0.5673618	total: 1m 7s	remaining: 3.7s
948:	learn: 0.5672606	total: 1m 7s	remaining: 3.63s
949:	learn: 0.5672333	total: 1m 7s	remaining: 3.56s
950:	learn: 0.5672122	total: 1m 7s	remaining: 3.49s
951:	learn: 0.5672005	total: 1m 7s	remaining: 3.42s
952:	learn: 0.5671820	total: 1m 7s	remaining: 3.35s
953:	learn: 0.

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.62      0.67      0.65       763
         1.0       0.63      0.57      0.60       728

   micro avg       0.62      0.62      0.62      1491
   macro avg       0.62      0.62      0.62      1491
weighted avg       0.62      0.62      0.62      1491

[[515 248]
 [313 415]]
Accuracy is  62.37424547283702
Time on model's work: 0.184 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.56      0.93      0.70       763
         1.0       0.76      0.22      0.35       728

   micro avg       0.59      0.59      0.59      1491
   macro avg       0.66      0.58      0.52      1491
weighted avg       0.66      0.59      0.53      1491

[[712  51]
 [565 163]]
Accuracy is  58.68544600938967
Time on model's work: 0.163 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.61      0.62      0.61       763
         1.0       0.59      0.58      0.58       728

   micro avg       0.60      0.60      0.60      1491
   macro avg       0.60      0.60      0.60      1491
weighted avg       0.60      0.60      0.60      1491

[[475 288]
 [308 420]]
Accuracy is  60.02682763246143
Time on model's work: 19.537 s
TOTAL TIME SPENT:  307.114 s


In [32]:
# TFFM sparse - works worse with sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)
# weight - optional / AdamOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:02<00:00, 17.89epoch/s]


accuracy: 0.6170355466130114
[[512 251]
 [320 408]]
              precision    recall  f1-score   support

         0.0       0.62      0.67      0.64       763
         1.0       0.62      0.56      0.59       728

   micro avg       0.62      0.62      0.62      1491
   macro avg       0.62      0.62      0.62      1491
weighted avg       0.62      0.62      0.62      1491



100%|██████████| 50/50 [00:02<00:00, 18.10epoch/s]


accuracy: 0.574111334674715
[[261 502]
 [133 595]]
              precision    recall  f1-score   support

         0.0       0.66      0.34      0.45       763
         1.0       0.54      0.82      0.65       728

   micro avg       0.57      0.57      0.57      1491
   macro avg       0.60      0.58      0.55      1491
weighted avg       0.60      0.57      0.55      1491



100%|██████████| 50/50 [00:02<00:00, 17.87epoch/s]


accuracy: 0.5197853789403085
[[117 646]
 [ 70 658]]
              precision    recall  f1-score   support

         0.0       0.63      0.15      0.25       763
         1.0       0.50      0.90      0.65       728

   micro avg       0.52      0.52      0.52      1491
   macro avg       0.57      0.53      0.45      1491
weighted avg       0.57      0.52      0.44      1491



100%|██████████| 50/50 [00:02<00:00, 18.27epoch/s]


accuracy: 0.5050301810865191
[[ 54 709]
 [ 29 699]]
              precision    recall  f1-score   support

         0.0       0.65      0.07      0.13       763
         1.0       0.50      0.96      0.65       728

   micro avg       0.51      0.51      0.51      1491
   macro avg       0.57      0.52      0.39      1491
weighted avg       0.58      0.51      0.38      1491



In [33]:
# weight - optional / FtrlOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:02<00:00, 17.59epoch/s]


accuracy: 0.5761234071093226
[[412 351]
 [281 447]]
              precision    recall  f1-score   support

         0.0       0.59      0.54      0.57       763
         1.0       0.56      0.61      0.59       728

   micro avg       0.58      0.58      0.58      1491
   macro avg       0.58      0.58      0.58      1491
weighted avg       0.58      0.58      0.58      1491



100%|██████████| 50/50 [00:02<00:00, 17.53epoch/s]


accuracy: 0.48826291079812206
[[  0 763]
 [  0 728]]


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       763
         1.0       0.49      1.00      0.66       728

   micro avg       0.49      0.49      0.49      1491
   macro avg       0.24      0.50      0.33      1491
weighted avg       0.24      0.49      0.32      1491



100%|██████████| 50/50 [00:02<00:00, 18.10epoch/s]


accuracy: 0.48826291079812206
[[  0 763]
 [  0 728]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       763
         1.0       0.49      1.00      0.66       728

   micro avg       0.49      0.49      0.49      1491
   macro avg       0.24      0.50      0.33      1491
weighted avg       0.24      0.49      0.32      1491



100%|██████████| 50/50 [00:02<00:00, 18.28epoch/s]


accuracy: 0.48826291079812206
[[  0 763]
 [  0 728]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       763
         1.0       0.49      1.00      0.66       728

   micro avg       0.49      0.49      0.49      1491
   macro avg       0.24      0.50      0.33      1491
weighted avg       0.24      0.49      0.32      1491



In [34]:
# KERAS
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=256)
score = model.evaluate(X_test, y_test, batch_size=256)
print(score)

Epoch 1/20
5961/5961 [==============================] - ETA: 7s - loss: 0.7270 - acc: 0.550 - ETA: 1s - loss: 0.7410 - acc: 0.529 - ETA: 1s - loss: 0.7400 - acc: 0.524 - ETA: 0s - loss: 0.7391 - acc: 0.518 - ETA: 0s - loss: 0.7422 - acc: 0.510 - ETA: 0s - loss: 0.7414 - acc: 0.507 - ETA: 0s - loss: 0.7421 - acc: 0.505 - ETA: 0s - loss: 0.7400 - acc: 0.507 - 1s 132us/step - loss: 0.7398 - acc: 0.5063
Epoch 2/20
5961/5961 [==============================] - ETA: 0s - loss: 0.6979 - acc: 0.535 - ETA: 0s - loss: 0.7250 - acc: 0.496 - ETA: 0s - loss: 0.7181 - acc: 0.506 - ETA: 0s - loss: 0.7198 - acc: 0.506 - ETA: 0s - loss: 0.7218 - acc: 0.503 - ETA: 0s - loss: 0.7206 - acc: 0.503 - ETA: 0s - loss: 0.7204 - acc: 0.503 - 0s 67us/step - loss: 0.7203 - acc: 0.5036
Epoch 3/20
5961/5961 [==============================] - ETA: 0s - loss: 0.7210 - acc: 0.511 - ETA: 0s - loss: 0.7145 - acc: 0.503 - ETA: 0s - loss: 0.7161 - acc: 0.503 - ETA: 0s - loss: 0.7165 - acc: 0.497 - ETA: 0s - loss: 0.7162 - 

## Nearmiss (version = 1) shows the best results

In [4]:
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours()
X_resampled_enn, y_resampled_enn = enn.fit_resample(features_list_array, labels_list_array)
print(sorted(Counter(y_resampled_enn).items()))

[(0.0, 38652), (1.0, 3726)]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled_enn, y_resampled_enn, random_state=35, test_size=0.2)

In [6]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96      7748
         1.0       0.75      0.32      0.45       728

   micro avg       0.93      0.93      0.93      8476
   macro avg       0.84      0.65      0.71      8476
weighted avg       0.92      0.93      0.92      8476

[[7669   79]
 [ 496  232]]
Accuracy is  93.21613968853232
Time on model's work: 5.333 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      7748
         1.0       0.92      0.07      0.13       728

   micro avg       0.92      0.92      0.92      8476
   macro avg       0.92      0.53      0.54      8476
weighted avg       0.92      0.92      0.89      8476

[[7744    4]
 [ 679   49]]
Accuracy is  91.9419537517697
Time on model's work: 308.262 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96      7748
         1.0       0.75      0.32      0.45       728

   micro avg       0.93      0.93      0.93      8476
   macro avg       0.85      0.66      0.71      8476
weighted avg       0.92      0.93      0.92      8476

[[7672   76]
 [ 495  233]]
Accuracy is  93.26333176026428
Time on model's work: 9.704 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      7748
         1.0       0.83      0.07      0.13       728

   micro avg       0.92      0.92      0.92      8476
   macro avg       0.88      0.53      0.54      8476
weighted avg       0.91      0.92      0.89      8476

[[7738   10]
 [ 678   50]]
Accuracy is  91.88296366210477
Time on model's work: 72.002 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96      7748
  

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

907:	learn: 0.1964215	total: 4m 58s	remaining: 30.2s
908:	learn: 0.1964021	total: 4m 58s	remaining: 29.9s
909:	learn: 0.1963979	total: 4m 58s	remaining: 29.5s
910:	learn: 0.1963696	total: 4m 58s	remaining: 29.2s
911:	learn: 0.1963628	total: 4m 59s	remaining: 28.9s
912:	learn: 0.1963617	total: 4m 59s	remaining: 28.5s
913:	learn: 0.1963543	total: 4m 59s	remaining: 28.2s
914:	learn: 0.1963526	total: 5m	remaining: 27.9s
915:	learn: 0.1963347	total: 5m	remaining: 27.6s
916:	learn: 0.1963109	total: 5m	remaining: 27.2s
917:	learn: 0.1963037	total: 5m 1s	remaining: 26.9s
918:	learn: 0.1962748	total: 5m 1s	remaining: 26.6s
919:	learn: 0.1962534	total: 5m 1s	remaining: 26.2s
920:	learn: 0.1962260	total: 5m 2s	remaining: 25.9s
921:	learn: 0.1962223	total: 5m 2s	remaining: 25.6s
922:	learn: 0.1962216	total: 5m 2s	remaining: 25.2s
923:	learn: 0.1962153	total: 5m 2s	remaining: 24.9s
924:	learn: 0.1962096	total: 5m 3s	remaining: 24.6s
925:	learn: 0.1962078	total: 5m 3s	remaining: 24.2s
926:	learn: 0.

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      7748
         1.0       0.73      0.09      0.17       728

   micro avg       0.92      0.92      0.92      8476
   macro avg       0.83      0.55      0.56      8476
weighted avg       0.91      0.92      0.89      8476

[[7723   25]
 [ 659   69]]
Accuracy is  91.93015573383671
Time on model's work: 1.057 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95      7748
         1.0       0.42      0.25      0.32       728

   micro avg       0.91      0.91      0.91      8476
   macro avg       0.68      0.61      0.63      8476
weighted avg       0.89      0.91      0.90      8476

[[7495  253]
 [ 544  184]]
Accuracy is  90.59697970740915
Time on model's work: 0.804 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95      7748
         1.0       0.51      0.13      0.21       728

   micro avg       0.91      0.91      0.91      8476
   macro avg       0.72      0.56      0.58      8476
weighted avg       0.89      0.91      0.89      8476

[[7659   89]
 [ 634   94]]
Accuracy is  91.47003303445021
Time on model's work: 119.171 s
TOTAL TIME SPENT:  2230.274 s


In [7]:
# TFFM sparse - works worse with sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)
# weight - optional / AdamOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:16<00:00,  3.36epoch/s]


accuracy: 0.9177678150070788
[[7662   86]
 [ 611  117]]
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      7748
         1.0       0.58      0.16      0.25       728

   micro avg       0.92      0.92      0.92      8476
   macro avg       0.75      0.57      0.60      8476
weighted avg       0.90      0.92      0.90      8476



100%|██████████| 50/50 [00:15<00:00,  3.44epoch/s]


accuracy: 0.9084473808400189
[[7495  253]
 [ 523  205]]
              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95      7748
         1.0       0.45      0.28      0.35       728

   micro avg       0.91      0.91      0.91      8476
   macro avg       0.69      0.62      0.65      8476
weighted avg       0.89      0.91      0.90      8476



100%|██████████| 50/50 [00:14<00:00,  3.44epoch/s]


accuracy: 0.8900424728645587
[[7231  517]
 [ 415  313]]
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94      7748
         1.0       0.38      0.43      0.40       728

   micro avg       0.89      0.89      0.89      8476
   macro avg       0.66      0.68      0.67      8476
weighted avg       0.90      0.89      0.89      8476



100%|██████████| 50/50 [00:14<00:00,  3.53epoch/s]


accuracy: 0.8671543180745634
[[6947  801]
 [ 325  403]]
              precision    recall  f1-score   support

         0.0       0.96      0.90      0.93      7748
         1.0       0.33      0.55      0.42       728

   micro avg       0.87      0.87      0.87      8476
   macro avg       0.65      0.73      0.67      8476
weighted avg       0.90      0.87      0.88      8476



In [8]:
# weight - optional / FtrlOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:14<00:00,  3.47epoch/s]


accuracy: 0.9141104294478528
[[7748    0]
 [ 728    0]]


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.91      1.00      0.96      7748
         1.0       0.00      0.00      0.00       728

   micro avg       0.91      0.91      0.91      8476
   macro avg       0.46      0.50      0.48      8476
weighted avg       0.84      0.91      0.87      8476



100%|██████████| 50/50 [00:14<00:00,  3.44epoch/s]


accuracy: 0.9137564889098632
[[7745    3]
 [ 728    0]]
              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95      7748
         1.0       0.00      0.00      0.00       728

   micro avg       0.91      0.91      0.91      8476
   macro avg       0.46      0.50      0.48      8476
weighted avg       0.84      0.91      0.87      8476



100%|██████████| 50/50 [00:14<00:00,  3.49epoch/s]


accuracy: 0.8856772062293534
[[7313  435]
 [ 534  194]]
              precision    recall  f1-score   support

         0.0       0.93      0.94      0.94      7748
         1.0       0.31      0.27      0.29       728

   micro avg       0.89      0.89      0.89      8476
   macro avg       0.62      0.61      0.61      8476
weighted avg       0.88      0.89      0.88      8476



100%|██████████| 50/50 [00:14<00:00,  3.41epoch/s]


accuracy: 0.8141812175554507
[[6442 1306]
 [ 269  459]]
              precision    recall  f1-score   support

         0.0       0.96      0.83      0.89      7748
         1.0       0.26      0.63      0.37       728

   micro avg       0.81      0.81      0.81      8476
   macro avg       0.61      0.73      0.63      8476
weighted avg       0.90      0.81      0.85      8476



In [9]:
# KERAS
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=256)
score = model.evaluate(X_test, y_test, batch_size=256)
print(score)

Epoch 1/20
33902/33902 [==============================] - ETA: 38s - loss: 0.8785 - acc: 0.40 - ETA: 9s - loss: 0.5812 - acc: 0.7008 - ETA: 7s - loss: 0.5273 - acc: 0.753 - ETA: 7s - loss: 0.5113 - acc: 0.769 - ETA: 6s - loss: 0.4750 - acc: 0.801 - ETA: 5s - loss: 0.4467 - acc: 0.823 - ETA: 4s - loss: 0.4322 - acc: 0.834 - ETA: 4s - loss: 0.4231 - acc: 0.841 - ETA: 4s - loss: 0.4115 - acc: 0.850 - ETA: 4s - loss: 0.4007 - acc: 0.857 - ETA: 4s - loss: 0.3965 - acc: 0.860 - ETA: 4s - loss: 0.3898 - acc: 0.864 - ETA: 3s - loss: 0.3832 - acc: 0.868 - ETA: 3s - loss: 0.3757 - acc: 0.872 - ETA: 3s - loss: 0.3692 - acc: 0.875 - ETA: 3s - loss: 0.3662 - acc: 0.877 - ETA: 3s - loss: 0.3623 - acc: 0.879 - ETA: 3s - loss: 0.3613 - acc: 0.879 - ETA: 3s - loss: 0.3591 - acc: 0.880 - ETA: 2s - loss: 0.3574 - acc: 0.882 - ETA: 2s - loss: 0.3532 - acc: 0.885 - ETA: 2s - loss: 0.3515 - acc: 0.886 - ETA: 2s - loss: 0.3477 - acc: 0.887 - ETA: 2s - loss: 0.3450 - acc: 0.889 - ETA: 2s - loss: 0.3426 - acc:

33902/33902 [==============================] - ETA: 2s - loss: 0.1991 - acc: 0.929 - ETA: 2s - loss: 0.2247 - acc: 0.917 - ETA: 2s - loss: 0.2241 - acc: 0.918 - ETA: 2s - loss: 0.2328 - acc: 0.914 - ETA: 2s - loss: 0.2343 - acc: 0.913 - ETA: 2s - loss: 0.2304 - acc: 0.914 - ETA: 2s - loss: 0.2321 - acc: 0.913 - ETA: 2s - loss: 0.2333 - acc: 0.912 - ETA: 2s - loss: 0.2362 - acc: 0.909 - ETA: 2s - loss: 0.2372 - acc: 0.909 - ETA: 2s - loss: 0.2366 - acc: 0.908 - ETA: 2s - loss: 0.2365 - acc: 0.908 - ETA: 2s - loss: 0.2368 - acc: 0.907 - ETA: 2s - loss: 0.2375 - acc: 0.908 - ETA: 2s - loss: 0.2370 - acc: 0.908 - ETA: 2s - loss: 0.2376 - acc: 0.907 - ETA: 2s - loss: 0.2353 - acc: 0.907 - ETA: 1s - loss: 0.2353 - acc: 0.907 - ETA: 1s - loss: 0.2346 - acc: 0.908 - ETA: 1s - loss: 0.2338 - acc: 0.908 - ETA: 1s - loss: 0.2325 - acc: 0.909 - ETA: 1s - loss: 0.2323 - acc: 0.909 - ETA: 1s - loss: 0.2324 - acc: 0.909 - ETA: 1s - loss: 0.2328 - acc: 0.909 - ETA: 1s - loss: 0.2320 - acc: 0.909 - ETA

33902/33902 [==============================] - ETA: 2s - loss: 0.1696 - acc: 0.929 - ETA: 2s - loss: 0.2551 - acc: 0.902 - ETA: 2s - loss: 0.2497 - acc: 0.905 - ETA: 2s - loss: 0.2398 - acc: 0.910 - ETA: 2s - loss: 0.2329 - acc: 0.912 - ETA: 2s - loss: 0.2282 - acc: 0.913 - ETA: 1s - loss: 0.2266 - acc: 0.914 - ETA: 1s - loss: 0.2253 - acc: 0.915 - ETA: 1s - loss: 0.2241 - acc: 0.915 - ETA: 1s - loss: 0.2283 - acc: 0.913 - ETA: 1s - loss: 0.2291 - acc: 0.913 - ETA: 1s - loss: 0.2257 - acc: 0.915 - ETA: 1s - loss: 0.2252 - acc: 0.916 - ETA: 1s - loss: 0.2253 - acc: 0.915 - ETA: 1s - loss: 0.2217 - acc: 0.917 - ETA: 1s - loss: 0.2221 - acc: 0.916 - ETA: 1s - loss: 0.2223 - acc: 0.916 - ETA: 1s - loss: 0.2209 - acc: 0.916 - ETA: 1s - loss: 0.2198 - acc: 0.917 - ETA: 1s - loss: 0.2180 - acc: 0.917 - ETA: 1s - loss: 0.2183 - acc: 0.917 - ETA: 1s - loss: 0.2195 - acc: 0.916 - ETA: 1s - loss: 0.2189 - acc: 0.916 - ETA: 1s - loss: 0.2183 - acc: 0.917 - ETA: 0s - loss: 0.2189 - acc: 0.916 - ETA

In [4]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled_smote_enn, y_resampled_smote_enn = smote_enn.fit_resample(features_list_array, labels_list_array)
print(sorted(Counter(y_resampled_smote_enn).items()))

[(0.0, 12603), (1.0, 15225)]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled_smote_enn, y_resampled_smote_enn, random_state=35, test_size=0.2)

In [6]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      2587
         1.0       0.98      0.99      0.98      2979

   micro avg       0.98      0.98      0.98      5566
   macro avg       0.98      0.98      0.98      5566
weighted avg       0.98      0.98      0.98      5566

[[2538   49]
 [  44 2935]]
Accuracy is  98.32914121451671
Time on model's work: 2.02 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.95      0.83      0.89      2587
         1.0       0.87      0.96      0.91      2979

   micro avg       0.90      0.90      0.90      5566
   macro avg       0.91      0.90      0.90      5566
weighted avg       0.91      0.90      0.90      5566

[[2149  438]
 [ 105 2874]]
Accuracy is  90.24434063959755
Time on model's work: 140.479 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98      2587
         1.0       0.98      0.99      0.99      2979

   micro avg       0.99      0.99      0.99      5566
   macro avg       0.99      0.98      0.99      5566
weighted avg       0.99      0.99      0.99      5566

[[2524   63]
 [  17 2962]]
Accuracy is  98.56270212001438
Time on model's work: 4.549 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.94      0.82      0.87      2587
         1.0       0.86      0.95      0.90      2979

   micro avg       0.89      0.89      0.89      5566
   macro avg       0.90      0.88      0.89      5566
weighted avg       0.89      0.89      0.89      5566

[[2109  478]
 [ 142 2837]]
Accuracy is  88.8609414301114
Time on model's work: 35.168 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97      2587
   

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

914:	learn: 0.0893955	total: 2m 56s	remaining: 16.4s
915:	learn: 0.0893610	total: 2m 56s	remaining: 16.2s
916:	learn: 0.0893508	total: 2m 56s	remaining: 16s
917:	learn: 0.0893421	total: 2m 57s	remaining: 15.8s
918:	learn: 0.0892201	total: 2m 57s	remaining: 15.6s
919:	learn: 0.0891807	total: 2m 57s	remaining: 15.4s
920:	learn: 0.0891573	total: 2m 57s	remaining: 15.2s
921:	learn: 0.0891464	total: 2m 57s	remaining: 15s
922:	learn: 0.0891228	total: 2m 57s	remaining: 14.8s
923:	learn: 0.0890563	total: 2m 58s	remaining: 14.7s
924:	learn: 0.0889708	total: 2m 58s	remaining: 14.5s
925:	learn: 0.0889651	total: 2m 58s	remaining: 14.3s
926:	learn: 0.0889507	total: 2m 58s	remaining: 14.1s
927:	learn: 0.0889378	total: 2m 58s	remaining: 13.9s
928:	learn: 0.0889330	total: 2m 59s	remaining: 13.7s
929:	learn: 0.0889219	total: 2m 59s	remaining: 13.5s
930:	learn: 0.0889113	total: 2m 59s	remaining: 13.3s
931:	learn: 0.0888994	total: 2m 59s	remaining: 13.1s
932:	learn: 0.0888901	total: 2m 59s	remaining: 12.

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.94      0.81      0.87      2587
         1.0       0.85      0.96      0.90      2979

   micro avg       0.89      0.89      0.89      5566
   macro avg       0.90      0.88      0.89      5566
weighted avg       0.90      0.89      0.89      5566

[[2103  484]
 [ 130 2849]]
Accuracy is  88.96873877111031
Time on model's work: 0.633 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.92      0.83      0.87      2587
         1.0       0.86      0.93      0.90      2979

   micro avg       0.88      0.88      0.88      5566
   macro avg       0.89      0.88      0.88      5566
weighted avg       0.89      0.88      0.88      5566

[[2143  444]
 [ 197 2782]]
Accuracy is  88.48365073661516
Time on model's work: 0.382 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.95      0.82      0.88      2587
         1.0       0.86      0.96      0.91      2979

   micro avg       0.90      0.90      0.90      5566
   macro avg       0.90      0.89      0.89      5566
weighted avg       0.90      0.90      0.89      5566

[[2126  461]
 [ 123 2856]]
Accuracy is  89.50772547610492
Time on model's work: 52.556 s
TOTAL TIME SPENT:  1062.091 s


In [8]:
# TFFM sparse - works worse with sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)
# weight - optional / AdamOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:09<00:00,  5.15epoch/s]


accuracy: 0.8979518505210204
[[2140  447]
 [ 121 2858]]
              precision    recall  f1-score   support

         0.0       0.95      0.83      0.88      2587
         1.0       0.86      0.96      0.91      2979

   micro avg       0.90      0.90      0.90      5566
   macro avg       0.91      0.89      0.90      5566
weighted avg       0.90      0.90      0.90      5566



100%|██████████| 50/50 [00:09<00:00,  5.18epoch/s]


accuracy: 0.8981315127560187
[[2088  499]
 [  68 2911]]
              precision    recall  f1-score   support

         0.0       0.97      0.81      0.88      2587
         1.0       0.85      0.98      0.91      2979

   micro avg       0.90      0.90      0.90      5566
   macro avg       0.91      0.89      0.90      5566
weighted avg       0.91      0.90      0.90      5566



100%|██████████| 50/50 [00:09<00:00,  5.16epoch/s]


accuracy: 0.8889687387711103
[[2020  567]
 [  51 2928]]
              precision    recall  f1-score   support

         0.0       0.98      0.78      0.87      2587
         1.0       0.84      0.98      0.90      2979

   micro avg       0.89      0.89      0.89      5566
   macro avg       0.91      0.88      0.89      5566
weighted avg       0.90      0.89      0.89      5566



100%|██████████| 50/50 [00:09<00:00,  5.16epoch/s]


accuracy: 0.8842975206611571
[[1978  609]
 [  35 2944]]
              precision    recall  f1-score   support

         0.0       0.98      0.76      0.86      2587
         1.0       0.83      0.99      0.90      2979

   micro avg       0.88      0.88      0.88      5566
   macro avg       0.91      0.88      0.88      5566
weighted avg       0.90      0.88      0.88      5566



In [9]:
# weight - optional / FtrlOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:10<00:00,  4.73epoch/s]


accuracy: 0.8512396694214877
[[2063  524]
 [ 304 2675]]
              precision    recall  f1-score   support

         0.0       0.87      0.80      0.83      2587
         1.0       0.84      0.90      0.87      2979

   micro avg       0.85      0.85      0.85      5566
   macro avg       0.85      0.85      0.85      5566
weighted avg       0.85      0.85      0.85      5566



100%|██████████| 50/50 [00:10<00:00,  5.01epoch/s]


accuracy: 0.8681279195113187
[[2043  544]
 [ 190 2789]]
              precision    recall  f1-score   support

         0.0       0.91      0.79      0.85      2587
         1.0       0.84      0.94      0.88      2979

   micro avg       0.87      0.87      0.87      5566
   macro avg       0.88      0.86      0.87      5566
weighted avg       0.87      0.87      0.87      5566



100%|██████████| 50/50 [00:10<00:00,  5.02epoch/s]


accuracy: 0.8607617678763924
[[1964  623]
 [ 152 2827]]
              precision    recall  f1-score   support

         0.0       0.93      0.76      0.84      2587
         1.0       0.82      0.95      0.88      2979

   micro avg       0.86      0.86      0.86      5566
   macro avg       0.87      0.85      0.86      5566
weighted avg       0.87      0.86      0.86      5566



100%|██████████| 50/50 [00:10<00:00,  4.99epoch/s]


accuracy: 0.8559108875314408
[[1894  693]
 [ 109 2870]]
              precision    recall  f1-score   support

         0.0       0.95      0.73      0.83      2587
         1.0       0.81      0.96      0.88      2979

   micro avg       0.86      0.86      0.86      5566
   macro avg       0.88      0.85      0.85      5566
weighted avg       0.87      0.86      0.85      5566



In [10]:
# KERAS
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=256)
score = model.evaluate(X_test, y_test, batch_size=256)
print(score)

Epoch 1/20
22262/22262 [==============================] - ETA: 22s - loss: 0.8499 - acc: 0.45 - ETA: 4s - loss: 0.7651 - acc: 0.5176 - ETA: 2s - loss: 0.7428 - acc: 0.535 - ETA: 2s - loss: 0.7325 - acc: 0.541 - ETA: 2s - loss: 0.7277 - acc: 0.544 - ETA: 1s - loss: 0.7165 - acc: 0.556 - ETA: 1s - loss: 0.7106 - acc: 0.560 - ETA: 1s - loss: 0.7063 - acc: 0.564 - ETA: 1s - loss: 0.7018 - acc: 0.568 - ETA: 1s - loss: 0.6943 - acc: 0.578 - ETA: 1s - loss: 0.6818 - acc: 0.591 - ETA: 1s - loss: 0.6740 - acc: 0.600 - ETA: 1s - loss: 0.6671 - acc: 0.609 - ETA: 0s - loss: 0.6612 - acc: 0.616 - ETA: 0s - loss: 0.6504 - acc: 0.629 - ETA: 0s - loss: 0.6422 - acc: 0.639 - ETA: 0s - loss: 0.6326 - acc: 0.650 - ETA: 0s - loss: 0.6261 - acc: 0.657 - ETA: 0s - loss: 0.6174 - acc: 0.666 - ETA: 0s - loss: 0.6095 - acc: 0.674 - ETA: 0s - loss: 0.6011 - acc: 0.683 - ETA: 0s - loss: 0.5960 - acc: 0.687 - ETA: 0s - loss: 0.5902 - acc: 0.694 - 2s 70us/step - loss: 0.5851 - acc: 0.6992
Epoch 2/20
22262/22262 [=

22262/22262 [==============================] - ETA: 1s - loss: 0.2732 - acc: 0.886 - ETA: 1s - loss: 0.3155 - acc: 0.881 - ETA: 1s - loss: 0.3229 - acc: 0.880 - ETA: 1s - loss: 0.3264 - acc: 0.875 - ETA: 1s - loss: 0.3140 - acc: 0.877 - ETA: 1s - loss: 0.3159 - acc: 0.876 - ETA: 1s - loss: 0.3166 - acc: 0.875 - ETA: 0s - loss: 0.3171 - acc: 0.875 - ETA: 0s - loss: 0.3140 - acc: 0.876 - ETA: 0s - loss: 0.3112 - acc: 0.879 - ETA: 0s - loss: 0.3116 - acc: 0.879 - ETA: 0s - loss: 0.3071 - acc: 0.881 - ETA: 0s - loss: 0.3076 - acc: 0.881 - ETA: 0s - loss: 0.3069 - acc: 0.882 - ETA: 0s - loss: 0.3054 - acc: 0.883 - ETA: 0s - loss: 0.3059 - acc: 0.882 - ETA: 0s - loss: 0.3061 - acc: 0.882 - ETA: 0s - loss: 0.3069 - acc: 0.882 - ETA: 0s - loss: 0.3064 - acc: 0.882 - ETA: 0s - loss: 0.3047 - acc: 0.882 - ETA: 0s - loss: 0.3033 - acc: 0.883 - ETA: 0s - loss: 0.3037 - acc: 0.883 - ETA: 0s - loss: 0.3034 - acc: 0.883 - ETA: 0s - loss: 0.3016 - acc: 0.884 - 1s 62us/step - loss: 0.3013 - acc: 0.8846

In [11]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_resampled_smote_tomek, y_resampled_smote_tomek = smote_tomek.fit_resample(features_list_array, labels_list_array)
print(sorted(Counter(y_resampled_smote_tomek).items()))

[(0.0, 18502), (1.0, 18502)]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled_smote_tomek, y_resampled_smote_tomek, random_state=35, test_size=0.2)

In [13]:
clfs = [
        ['RandomForestClassifier', RandomForestClassifier()],
        ['GradientBoostingClassifier', GradientBoostingClassifier()],
        ['ExtraTreesClassifier', ExtraTreesClassifier()],
        ['AdaBoostClassifier', AdaBoostClassifier()],
        ['BaggingClassifier', BaggingClassifier()],
        ['DecisionTreeClassifier', DecisionTreeClassifier()],
        ['MLPClassifier', MLPClassifier()],
        ['XGBClassifier', XGBClassifier()],
        ['CatBoostClassifier', CatBoostClassifier()],
        ['LogisticRegression', LogisticRegression()],
        ['SGDClassifier', SGDClassifier()],
        ['TFFMClassifier', TFFMClassifier()]
       ]
t = time()
for name, clf in clfs:
    t0 = time()
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print('Accuracy is ', accuracy_score(y_test, prediction)*100)
    print ("Time on model's work:", round(time()-t0, 3), "s")
    print('='*100)
print ("TOTAL TIME SPENT: ", round(time()-t, 3), "s")

C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94      3781
         1.0       0.95      0.92      0.94      3620

   micro avg       0.94      0.94      0.94      7401
   macro avg       0.94      0.94      0.94      7401
weighted avg       0.94      0.94      0.94      7401

[[3609  172]
 [ 282 3338]]
Accuracy is  93.86569382515876
Time on model's work: 3.074 s
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       0.91      0.78      0.84      3781
         1.0       0.80      0.92      0.86      3620

   micro avg       0.85      0.85      0.85      7401
   macro avg       0.86      0.85      0.85      7401
weighted avg       0.86      0.85      0.85      7401

[[2960  821]
 [ 286 3334]]
Accuracy is  85.04256181597081
Time on model's work: 214.916 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94      3781
         1.0       0.95      0.93      0.94      3620

   micro avg       0.94      0.94      0.94      7401
   macro avg       0.94      0.94      0.94      7401
weighted avg       0.94      0.94      0.94      7401

[[3595  186]
 [ 244 3376]]
Accuracy is  94.18997432779355
Time on model's work: 6.802 s
AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.88      0.73      0.80      3781
         1.0       0.76      0.89      0.82      3620

   micro avg       0.81      0.81      0.81      7401
   macro avg       0.82      0.81      0.81      7401
weighted avg       0.82      0.81      0.81      7401

[[2753 1028]
 [ 384 3236]]
Accuracy is  80.92149709498716
Time on model's work: 47.91 s
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.92      0.95      0.94      3781
   

97:	learn: 0.3020851	total: 33.1s	remaining: 5m 4s
98:	learn: 0.3014498	total: 33.4s	remaining: 5m 3s
99:	learn: 0.3005859	total: 33.7s	remaining: 5m 3s
100:	learn: 0.2992419	total: 34s	remaining: 5m 2s
101:	learn: 0.2989841	total: 34.3s	remaining: 5m 1s
102:	learn: 0.2988311	total: 34.5s	remaining: 5m
103:	learn: 0.2983429	total: 34.8s	remaining: 4m 59s
104:	learn: 0.2981288	total: 35.1s	remaining: 4m 58s
105:	learn: 0.2976077	total: 35.4s	remaining: 4m 58s
106:	learn: 0.2970635	total: 35.7s	remaining: 4m 57s
107:	learn: 0.2963908	total: 35.9s	remaining: 4m 56s
108:	learn: 0.2948922	total: 36.3s	remaining: 4m 56s
109:	learn: 0.2944404	total: 36.6s	remaining: 4m 55s
110:	learn: 0.2941960	total: 36.8s	remaining: 4m 55s
111:	learn: 0.2934196	total: 37.1s	remaining: 4m 54s
112:	learn: 0.2928920	total: 37.5s	remaining: 4m 53s
113:	learn: 0.2921070	total: 37.7s	remaining: 4m 53s
114:	learn: 0.2916729	total: 38s	remaining: 4m 52s
115:	learn: 0.2914540	total: 38.3s	remaining: 4m 52s
116:	lear

...

869:	learn: 0.1636642	total: 4m 8s	remaining: 37.1s
870:	learn: 0.1636578	total: 4m 8s	remaining: 36.8s
871:	learn: 0.1636540	total: 4m 8s	remaining: 36.5s
872:	learn: 0.1636495	total: 4m 9s	remaining: 36.2s
873:	learn: 0.1636438	total: 4m 9s	remaining: 35.9s
874:	learn: 0.1636221	total: 4m 9s	remaining: 35.6s
875:	learn: 0.1636195	total: 4m 9s	remaining: 35.4s
876:	learn: 0.1636097	total: 4m 10s	remaining: 35.1s
877:	learn: 0.1636077	total: 4m 10s	remaining: 34.8s
878:	learn: 0.1635692	total: 4m 10s	remaining: 34.5s
879:	learn: 0.1635662	total: 4m 10s	remaining: 34.2s
880:	learn: 0.1634992	total: 4m 11s	remaining: 33.9s
881:	learn: 0.1634632	total: 4m 11s	remaining: 33.6s
882:	learn: 0.1634389	total: 4m 11s	remaining: 33.4s
883:	learn: 0.1634309	total: 4m 11s	remaining: 33.1s
884:	learn: 0.1634199	total: 4m 12s	remaining: 32.8s
885:	learn: 0.1634072	total: 4m 12s	remaining: 32.5s
886:	learn: 0.1634004	total: 4m 12s	remaining: 32.2s
887:	learn: 0.1633964	total: 4m 12s	remaining: 31.9s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
              precision    recall  f1-score   support

         0.0       0.90      0.70      0.79      3781
         1.0       0.75      0.92      0.82      3620

   micro avg       0.81      0.81      0.81      7401
   macro avg       0.82      0.81      0.81      7401
weighted avg       0.82      0.81      0.81      7401

[[2658 1123]
 [ 298 3322]]
Accuracy is  80.79989190649913
Time on model's work: 0.92 s


C:\Users\maksim.parats\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
              precision    recall  f1-score   support

         0.0       0.93      0.62      0.74      3781
         1.0       0.70      0.95      0.81      3620

   micro avg       0.78      0.78      0.78      7401
   macro avg       0.81      0.78      0.77      7401
weighted avg       0.82      0.78      0.77      7401

[[2337 1444]
 [ 189 3431]]
Accuracy is  77.93541413322525
Time on model's work: 0.494 s
TFFMClassifier
              precision    recall  f1-score   support

         0.0       0.91      0.70      0.79      3781
         1.0       0.75      0.92      0.83      3620

   micro avg       0.81      0.81      0.81      7401
   macro avg       0.83      0.81      0.81      7401
weighted avg       0.83      0.81      0.81      7401

[[2662 1119]
 [ 274 3346]]
Accuracy is  81.17821915957303
Time on model's work: 63.489 s
TOTAL TIME SPENT:  1247.359 s


In [14]:
# TFFM sparse - works worse with sparse
# only CSR format supported
X_train_sparse = sp.csr_matrix(X_train)
X_test_sparse = sp.csr_matrix(X_test)
# weight - optional / AdamOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:13<00:00,  3.92epoch/s]


accuracy: 0.8197540872855019
[[2698 1083]
 [ 251 3369]]
              precision    recall  f1-score   support

         0.0       0.91      0.71      0.80      3781
         1.0       0.76      0.93      0.83      3620

   micro avg       0.82      0.82      0.82      7401
   macro avg       0.84      0.82      0.82      7401
weighted avg       0.84      0.82      0.82      7401



100%|██████████| 50/50 [00:13<00:00,  3.84epoch/s]


accuracy: 0.8079989190649912
[[2497 1284]
 [ 137 3483]]
              precision    recall  f1-score   support

         0.0       0.95      0.66      0.78      3781
         1.0       0.73      0.96      0.83      3620

   micro avg       0.81      0.81      0.81      7401
   macro avg       0.84      0.81      0.80      7401
weighted avg       0.84      0.81      0.80      7401



100%|██████████| 50/50 [00:14<00:00,  3.60epoch/s]


accuracy: 0.8048912309147412
[[2420 1361]
 [  83 3537]]
              precision    recall  f1-score   support

         0.0       0.97      0.64      0.77      3781
         1.0       0.72      0.98      0.83      3620

   micro avg       0.80      0.80      0.80      7401
   macro avg       0.84      0.81      0.80      7401
weighted avg       0.85      0.80      0.80      7401



100%|██████████| 50/50 [00:13<00:00,  3.83epoch/s]


accuracy: 0.7966491014727739
[[2347 1434]
 [  71 3549]]
              precision    recall  f1-score   support

         0.0       0.97      0.62      0.76      3781
         1.0       0.71      0.98      0.83      3620

   micro avg       0.80      0.80      0.80      7401
   macro avg       0.84      0.80      0.79      7401
weighted avg       0.84      0.80      0.79      7401



In [15]:
# weight - optional / FtrlOptimizer
pos_class_weight = list(map(float,range(1, 5)))
for weight in pos_class_weight:
    model = TFFMClassifier(
        order=2,
        pos_class_weight=weight,
        rank=10, 
        optimizer=tf.train.FtrlOptimizer(0.01, l2_regularization_strength=0.001, l2_shrinkage_regularization_strength=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        #log_dir='./tmp/logs',
        #verbose=1,
        seed=42
    )
    model.fit(X_train_sparse, y_train, show_progress=True)
    predictions = model.predict(X_test_sparse)
    print('accuracy: {}'.format(accuracy_score(y_test, predictions)))
    # this will close tf.Session and free resources
    print(confusion_matrix(y_test,predictions)) 
    print(classification_report(y_test, predictions))
    model.destroy()

100%|██████████| 50/50 [00:13<00:00,  3.77epoch/s]


accuracy: 0.7419267666531549
[[2305 1476]
 [ 434 3186]]
              precision    recall  f1-score   support

         0.0       0.84      0.61      0.71      3781
         1.0       0.68      0.88      0.77      3620

   micro avg       0.74      0.74      0.74      7401
   macro avg       0.76      0.74      0.74      7401
weighted avg       0.76      0.74      0.74      7401



100%|██████████| 50/50 [00:13<00:00,  3.83epoch/s]


accuracy: 0.7592217267936765
[[2288 1493]
 [ 289 3331]]
              precision    recall  f1-score   support

         0.0       0.89      0.61      0.72      3781
         1.0       0.69      0.92      0.79      3620

   micro avg       0.76      0.76      0.76      7401
   macro avg       0.79      0.76      0.75      7401
weighted avg       0.79      0.76      0.75      7401



100%|██████████| 50/50 [00:13<00:00,  3.81epoch/s]


accuracy: 0.7588163761653831
[[2218 1563]
 [ 222 3398]]
              precision    recall  f1-score   support

         0.0       0.91      0.59      0.71      3781
         1.0       0.68      0.94      0.79      3620

   micro avg       0.76      0.76      0.76      7401
   macro avg       0.80      0.76      0.75      7401
weighted avg       0.80      0.76      0.75      7401



100%|██████████| 50/50 [00:13<00:00,  3.85epoch/s]


accuracy: 0.7471963248209701
[[2071 1710]
 [ 161 3459]]
              precision    recall  f1-score   support

         0.0       0.93      0.55      0.69      3781
         1.0       0.67      0.96      0.79      3620

   micro avg       0.75      0.75      0.75      7401
   macro avg       0.80      0.75      0.74      7401
weighted avg       0.80      0.75      0.74      7401



In [16]:
# KERAS
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=256)
score = model.evaluate(X_test, y_test, batch_size=256)
print(score)

Epoch 1/20
29603/29603 [==============================] - ETA: 31s - loss: 0.7649 - acc: 0.53 - ETA: 6s - loss: 0.7570 - acc: 0.5026 - ETA: 4s - loss: 0.7446 - acc: 0.516 - ETA: 3s - loss: 0.7357 - acc: 0.526 - ETA: 3s - loss: 0.7351 - acc: 0.529 - ETA: 2s - loss: 0.7297 - acc: 0.533 - ETA: 2s - loss: 0.7244 - acc: 0.535 - ETA: 2s - loss: 0.7201 - acc: 0.539 - ETA: 1s - loss: 0.7121 - acc: 0.549 - ETA: 1s - loss: 0.7040 - acc: 0.559 - ETA: 1s - loss: 0.6979 - acc: 0.567 - ETA: 1s - loss: 0.6923 - acc: 0.573 - ETA: 1s - loss: 0.6872 - acc: 0.581 - ETA: 1s - loss: 0.6809 - acc: 0.589 - ETA: 1s - loss: 0.6759 - acc: 0.595 - ETA: 0s - loss: 0.6719 - acc: 0.601 - ETA: 0s - loss: 0.6668 - acc: 0.608 - ETA: 0s - loss: 0.6614 - acc: 0.615 - ETA: 0s - loss: 0.6568 - acc: 0.620 - ETA: 0s - loss: 0.6537 - acc: 0.624 - ETA: 0s - loss: 0.6505 - acc: 0.628 - ETA: 0s - loss: 0.6467 - acc: 0.632 - ETA: 0s - loss: 0.6431 - acc: 0.636 - ETA: 0s - loss: 0.6404 - acc: 0.640 - ETA: 0s - loss: 0.6388 - acc:

29603/29603 [==============================] - ETA: 1s - loss: 0.4074 - acc: 0.812 - ETA: 1s - loss: 0.4527 - acc: 0.785 - ETA: 1s - loss: 0.4578 - acc: 0.784 - ETA: 1s - loss: 0.4599 - acc: 0.778 - ETA: 1s - loss: 0.4522 - acc: 0.784 - ETA: 1s - loss: 0.4480 - acc: 0.788 - ETA: 1s - loss: 0.4445 - acc: 0.790 - ETA: 1s - loss: 0.4441 - acc: 0.791 - ETA: 0s - loss: 0.4476 - acc: 0.789 - ETA: 0s - loss: 0.4489 - acc: 0.788 - ETA: 0s - loss: 0.4516 - acc: 0.786 - ETA: 0s - loss: 0.4529 - acc: 0.785 - ETA: 0s - loss: 0.4535 - acc: 0.785 - ETA: 0s - loss: 0.4544 - acc: 0.785 - ETA: 0s - loss: 0.4553 - acc: 0.783 - ETA: 0s - loss: 0.4548 - acc: 0.784 - ETA: 0s - loss: 0.4543 - acc: 0.785 - ETA: 0s - loss: 0.4543 - acc: 0.785 - ETA: 0s - loss: 0.4538 - acc: 0.786 - ETA: 0s - loss: 0.4541 - acc: 0.786 - ETA: 0s - loss: 0.4547 - acc: 0.786 - ETA: 0s - loss: 0.4553 - acc: 0.786 - ETA: 0s - loss: 0.4542 - acc: 0.787 - ETA: 0s - loss: 0.4547 - acc: 0.786 - ETA: 0s - loss: 0.4537 - acc: 0.787 - 1s 

## RESULTS BEST SAMPLERS AND MODELS
### 1. Sampler 'SMOTEEN'. Models - RandomForest, ExtraTrees, MLP
### 2. Sampler 'Nearmiss(version = 1)'. Models - GradientBoosting, XGB, TFFM